# Homework 1 (Total Points: 250) <a class="anchor" id="top"></a>


**Submission instructions**:
- The cells with the `# YOUR CODE HERE` denote that these sections are graded and you need to add your implementation.
- For Part 1: You can use the `nltk`, `NumPy`, and `matplotlib` libraries here. Other libraries, e.g., `gensim` or `scikit-learn`, may not be used. For Part 2: `gensim` is allowed in addition to the imported libraries in the next code cell
- Please use Python 3.6.5 and `pip install -r requirements.txt` to avoid version issues.
- The notebook you submit has to have the student ids, separated by underscores (E.g., `12341234_12341234_12341234_hw1.ipynb`).
- This will be parsed by a regexp, **so please double check your filename**.
- Only one member of each group has to submit the file (**please do not compress the .ipynb file when you will submit it**) to canvas.
- **Make sure to check that your notebook runs before submission**. A quick way to do this is to restart the kernel and run all the cells.  
- Do not change the number of arugments in the given functions.
- **Please do not delete/add new cells**. Removing cells **will** lead to grade deduction. 
- Note, that you are not allowed to use Google Colab.


**Learning Goals**:
- [Part 1, Term-based matching](#part1) (165 points):
    - Learn how to load a dataset and process it.
    - Learn how to implement several standard IR methods (TF-IDF, BM25, QL) and understand their weaknesses & strengths.
    - Learn how to evaluate IR methods.
- [Part 2, Semantic-based matching](#part2) (85 points):
    - Learn how to implement vector-space retrieval methods (LSI, LDA).
    - Learn how to use LSI and LDA for re-ranking.

    
**Resources**: 
- **Part 1**: Sections 2.3, 4.1, 4.2, 4.3, 5.3, 5.6, 5.7, 6.2, 7, 8 of [Search Engines: Information Retrieval in Practice](https://ciir.cs.umass.edu/downloads/SEIRiP.pdf)
- **Part 2**: [LSI - Chapter 18](https://nlp.stanford.edu/IR-book/pdf/18lsi.pdf) from [Introduction to Information Retrieval](https://nlp.stanford.edu/IR-book/) book and the [original LDA paper](https://jmlr.org/papers/volume3/blei03a/blei03a.pdf)

In [1]:
# imports 
# TODO: Ensure that no additional library is imported in the notebook. 
# TODO: Only the standard library and the following libraries are allowed:
# TODO: You can also use unlisted classes from these libraries or standard libraries (such as defaultdict, Counter, ...).

import os
import zipfile
from functools import partial

import nltk
import requests
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

from ipywidgets import widgets
from IPython.display import display, HTML
#from IPython.html import widgets
from collections import namedtuple

%matplotlib inline


# Part 1: Term-based Matching (165 points) <a class="anchor" id="part1"></a>

[Back to top](#top)

In the first part, we will learn the basics of IR from loading and preprocessing the material, to implementing some well known search algorithms, to evaluating the ranking performance of the implemented algorithms. We will be using the CACM dataset throughout the assignment. The CACM dataset is a collection of titles and abstracts from the journal CACM (Communication of the ACM).

Table of contents:
- [Section 1: Text Processing](#text_processing) (5 points)
- [Section 2: Indexing](#indexing) (10 points)
- [Section 3: Ranking](#ranking) (80 points)
- [Section 4: Evaluation](#evaluation) (40 points)
- [Section 5: Analysis](#analysis) (30 points)


---
## Section 1: Text Processing (5 points)<a class="anchor" id="text_processing"></a>

[Back to Part 1](#part1)

In this section, we will load the dataset and learn how to clean up the data to make it usable for an IR system. 
First, go through the implementation of the following functions:
- `read_cacm_docs`: Reads in the CACM documents.
- `read_queries`: Reads in the CACM queries.
- `load_stopwords`: Loads the stopwords.

The points of this section are earned for the following implementations:
- `tokenize` (3 points): Tokenizes the input text.
- `stem_token` (2 points): Stems the given token. 

We are using the [CACM dataset](http://ir.dcs.gla.ac.uk/resources/test_collections/cacm/), which is a small, classic IR dataset, composed of a collection of titles and abstracts from the journal CACM. It comes with relevance judgements for queries, so we can evaluate our IR system. 


---
### 1.1 Read the CACM documents


The following cell downloads the dataset and unzips it to a local directory.

In [2]:
def download_dataset():
    folder_path = os.environ.get("IR1_DATA_PATH")
    if not folder_path:
        folder_path = "./datasets/"
    os.makedirs(folder_path, exist_ok=True)
    
    file_location = os.path.join(folder_path, "cacm.zip")
    
    # download file if it doesn't exist
    if not os.path.exists(file_location):
        
        url = "https://surfdrive.surf.nl/files/index.php/s/M0FGJpX2p8wDwxR/download"

        with open(file_location, "wb") as handle:
            print(f"Downloading file from {url} to {file_location}")
            response = requests.get(url, stream=True)
            for data in tqdm(response.iter_content()):
                handle.write(data)
            print("Finished downloading file")
    
    if not os.path.exists(os.path.join(folder_path, "train.txt")):
        
        # unzip file
        with zipfile.ZipFile(file_location, 'r') as zip_ref:
            zip_ref.extractall(folder_path)
        
download_dataset()

---

You can see a brief description of each file in the dataset by looking at the README file:

In [3]:
##### Read the README file 
with open ("./datasets/README","r") as file:
    readme = file.read()
    print(readme)
#####

Files in this directory with sizes:
          0 Jun 19 21:01 README

    2187734 Jun 19 20:55 cacm.all              text of documents
        626 Jun 19 20:58 cite.info             key to citation info
                                                (the X sections in cacm.all)
       2668 Jun 19 20:55 common_words           stop words used by smart
       2194 Jun 19 20:55 make_coll*             shell script to make collection
       1557 Jun 19 20:55 make_coll_term*        ditto (both useless without
                                                smart system)
       9948 Jun 19 20:55 qrels.text             relation giving
                                                    qid did 0 0
                                                to indicate dument did is
                                                relevant to query qid
      13689 Jun 19 20:55 query.text             Original text of the query



---
We are interested in 4 files:
- `cacm.all` : Contains the text for all documents. Note that some documents do not have abstracts available
- `query.text` : The text of all queries
- `qrels.text` : The relevance judgements
- `common_words` : A list of common words. This may be used as a collection of stopwords

In [4]:
##### The first 45 lines of the CACM dataset forms the first record
# We are interested only in 3 fields. 
# 1. the '.I' field, which is the document id
# 2. the '.T' field (the title) and
# 3. the '.W' field (the abstract, which may be absent)
with open ("./datasets/cacm.all","r") as file:
    cacm_all = "".join(file.readlines()[:45])
    print(cacm_all)
#####

.I 1
.T
Preliminary Report-International Algebraic Language
.B
CACM December, 1958
.A
Perlis, A. J.
Samelson,K.
.N
CA581203 JB March 22, 1978  8:28 PM
.X
100	5	1
123	5	1
164	5	1
1	5	1
1	5	1
1	5	1
205	5	1
210	5	1
214	5	1
1982	5	1
398	5	1
642	5	1
669	5	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
165	6	1
196	6	1
196	6	1
1273	6	1
1883	6	1
324	6	1
43	6	1
53	6	1
91	6	1
410	6	1
3184	6	1



---

The following function reads the `cacm.all` file. Note that each document has a variable number of lines. The `.I` field denotes a new document:

In [5]:
def read_cacm_docs(root_folder = "./datasets/"):
    """
        Reads in the CACM documents. The dataset is assumed to be in the folder "./datasets/" by default
        Returns: A list of 2-tuples: (doc_id, document), where 'document' is a single string created by 
            appending the title and abstract (separated by a "\n"). 
            In case the record doesn't have an abstract, the document is composed only by the title
    """
    with open(os.path.join(root_folder, "cacm.all")) as reader:
        lines = reader.readlines()
    
    doc_id, title, abstract = None, None, None
    
    docs = []
    line_idx = 0
    while line_idx < len(lines):
        line = lines[line_idx]
        if line.startswith(".I"):
            if doc_id is not None:
                docs.append((doc_id, title, abstract))
                doc_id, title, abstract = None, None, None
            
            doc_id = line.split()[-1]
            line_idx += 1
        elif line.startswith(".T"):
            # start at next line
            line_idx += 1
            temp_lines = []
            # read till next '.'
            while not lines[line_idx].startswith("."):
                temp_lines.append(lines[line_idx].strip("\n"))
                line_idx += 1
            title = "\n".join(temp_lines).strip("\n")
        elif line.startswith(".W"):
            # start at next line
            line_idx += 1
            temp_lines = []
            # read till next '.'
            while not lines[line_idx].startswith("."):
                temp_lines.append(lines[line_idx].strip("\n"))
                line_idx += 1
            abstract = "\n".join(temp_lines).strip("\n")
        else:
            line_idx += 1
    
    docs.append((doc_id, title, abstract))
    
    p_docs = []
    for (did, t, a) in docs:
        if a is None:
            a = ""
        p_docs.append((did, t + "\n" + a))
    return p_docs


In [6]:
##### Function check
docs = read_cacm_docs()

assert isinstance(docs, list)
assert len(docs) == 3204, "There should be exactly 3204 documents"

unzipped_docs = list(zip(*docs))
assert np.sum(np.array(list(map(int,unzipped_docs[0])))) == 5134410

##### 

---
### 1.2 Read the CACM queries

Next, let us read the queries. They are formatted similarly:

In [7]:
##### The first 15 lines of 'query.text' has 2 queries
# We are interested only in 2 fields. 
# 1. the '.I' - the query id
# 2. the '.W' - the query
!head -15 ./datasets/query.text
#####

.I 1
.W
 What articles exist which deal with TSS (Time Sharing System), an
operating system for IBM computers?
.N
 1. Richard Alexander, Comp Serv, Langmuir Lab (TSS)
 
.I 2
.W
 I am interested in articles written either by Prieve or Udo Pooch
.A
Prieve, B.
Pooch, U.
.N
 2. Richard Alexander, Comp Serv, Langmuir Lab (author = Pooch or Prieve)


---

The following function reads the `query.text` file:

In [8]:
def read_queries(root_folder = "./datasets/"):
    """
        Reads in the CACM queries. The dataset is assumed to be in the folder "./datasets/" by default
        Returns: A list of 2-tuples: (query_id, query)
    """
    with open(os.path.join(root_folder, "query.text")) as reader:
        lines = reader.readlines()
    
    query_id, query = None, None
    
    queries = []
    line_idx = 0
    while line_idx < len(lines):
        line = lines[line_idx]
        if line.startswith(".I"):
            if query_id is not None:
                queries.append((query_id, query))
                query_id, query = None, None
    
            query_id = line.split()[-1]
            line_idx += 1
        elif line.startswith(".W"):
            # start at next line
            line_idx += 1
            temp_lines = []
            # read till next '.'
            while not lines[line_idx].startswith("."):
                temp_lines.append(lines[line_idx].strip("\n"))
                line_idx += 1
            query = "\n".join(temp_lines).strip("\n")
        else:
            line_idx += 1
    
    queries.append((query_id, query))
    return queries


In [9]:
##### Function check
queries = read_queries()

assert isinstance(queries, list)
assert len(queries) == 64 and all([q[1] is not None for q in queries]), "There should be exactly 64 queries"

unzipped_queries = list(zip(*queries))
assert np.sum(np.array(list(map(int,unzipped_queries[0])))) == 2080

##### 

---
### 1.3 Read the stop words

We use the common words stored in `common_words`:

In [10]:
##### Read the stop words file 
!head ./datasets/common_words
##### Read the README file 

a
about
above
accordingly
across
after
afterwards
again
against
all


---

The following function reads the `common_words` file (For better coverage, we try to keep them in lowercase):

In [11]:
def load_stopwords(root_folder = "./datasets/"):
    """
        Loads the stopwords. The dataset is assumed to be in the folder "./datasets/" by default
        Output: A set of stopwords
    """
    with open(os.path.join(root_folder, "common_words")) as reader:
        lines = reader.readlines()
    stopwords = set([l.strip().lower() for l in lines])
    return stopwords


In [12]:
##### Function check
stopwords = load_stopwords()

assert isinstance(stopwords, set)
assert len(stopwords) == 428, "There should be exactly 428 stop words"

assert np.sum(np.array(list(map(len,stopwords)))) == 2234

##### 


---
### 1.4 Tokenization (3 points)

We can now write some basic text processing functions. 
A first step is to tokenize the text. 

**Note**: Use the  `WordPunctTokenizer` available in the `nltk` library:

In [13]:
# TODO: Implement this! (4 points)
def tokenize(text):
    """
        Tokenizes the input text. Use the WordPunctTokenizer
        Input: text - a string
        Output: a list of tokens
    """
    
    tk = nltk.WordPunctTokenizer()
    
    return tk.tokenize(text) 

In [14]:
##### Function check
text = "the quick brown fox jumps over the lazy dog"
tokens = tokenize(text)

assert isinstance(tokens, list)
assert len(tokens) == 9

print(tokens)
# output: ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
#####

['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']


---
### 1.5 Stemming (2 points)

Write a function to stem tokens. 
Again, you can use the nltk library for this:

In [15]:
# TODO: Implement this! (3 points)
def stem_token(token):
    """
        Stems the given token using the PorterStemmer from the nltk library
        Input: a single token
        Output: the stem of the token
    """
    stemmer = nltk.PorterStemmer()
    
    return stemmer.stem(token)

In [16]:
##### Function check

assert stem_token('owned') == 'own'
assert stem_token('itemization') == 'item'
#####

---
### 1.6 Summary

The following function puts it all together. Given an input string, this functions tokenizes and processes it according to the flags that you set.

In [17]:
#### Putting it all together
def process_text(text, stem=False, remove_stopwords=False, lowercase_text=False):
    
    tokens = []
    for token in tokenize(text):
        if remove_stopwords and token.lower() in stopwords:
            continue
        if stem:
            token = stem_token(token)
        if lowercase_text:
            token = token.lower()
        tokens.append(token)

    return tokens
#### 

---

Let's create two sets of preprocessed documents.
We can process the documents and queries according to these two configurations:

In [18]:
# In this configuration:
# Don't preprocess the text, except to tokenize 
config_1 = {
  "stem": False,
  "remove_stopwords" : False,
  "lowercase_text": True
} 


# In this configuration:
# Preprocess the text, stem and remove stopwords
config_2 = {
  "stem": True,
  "remove_stopwords" : True,
  "lowercase_text": True, 
} 

####
doc_repr_1 = []
doc_repr_2 = []
for (doc_id, document) in docs:
    doc_repr_1.append((doc_id, process_text(document, **config_1)))
    doc_repr_2.append((doc_id, process_text(document, **config_2)))

####

--- 

## Section 2: Indexing (10 points)<a class="anchor" id="indexing"></a>

[Back to Part 1](#part1)



A retrieval function usually takes in a query document pair, and scores a query against a document.  Our document set is quite small - just a few thousand documents. However, consider a web-scale dataset with a few million documents. In such a scenario, it would become infeasible to score every query and document pair. In such a case, we can build an inverted index. From Wikipedia:

> ... , an inverted index (also referred to as a postings file or inverted file) is a database index storing a mapping from content, such as words or numbers, to its locations in a table, .... The purpose of an inverted index is to allow fast full-text searches, at a cost of increased processing when a document is added to the database. ...


Consider a simple inverted index, which maps from word to document. This can improve the performance of a retrieval system significantly. In this assignment, we consider a *simple* inverted index, which maps a word to a set of documents. In practice, however, more complex indices might be used.  


### 2.1 Term Frequency-index (10 points)
In this assignment, we will be using an index created in memory since our dataset is tiny. To get started, build a simple index that maps each `token` to a list of `(doc_id, count)` where `count` is the count of the `token` in `doc_id`.
For consistency, build this index using a python dictionary.
    
Now, implement a function to build an index:

In [19]:
from collections import Counter, defaultdict

# TODO: Implement this! (10 points)
def build_tf_index(documents):
    """
        Build an inverted index (with counts). The output is a dictionary which takes in a token
        and returns a list of (doc_id, count) where 'count' is the count of the 'token' in 'doc_id'
        Input: a list of documents - (doc_id, tokens) 
        Output: An inverted index implemented within a pyhton dictionary: [token] -> [(doc_id, token_count)]
    """
    # YOUR CODE HERE
    
    # retrieve all unique words and create tuples (doc_id, {word: count})
    words = list(set([word for document in documents for word in document[1]]))
    document = [(doc[0], Counter(doc[1])) for doc in documents] 
    
    inv_index = defaultdict(list)
    
    for word in words:
        # count is a dictionary with tokens as keys and the tfs as values
        # only append if a word is present in count dict
        inv_index[word].append([(doc_id, count[word]) for doc_id, count in document if word in count])
            
    return inv_index

---
Now we can build indexed documents and preprocess the queries based on the two configurations:

In [20]:
#### Indexed documents based on the two configs

# Create the 2 indices
tf_index_1 = build_tf_index(doc_repr_1)
tf_index_2 = build_tf_index(doc_repr_2)

# This function returns the tf_index of the corresponding config
def get_index(index_set):
    assert index_set in {1, 2}
    return {
        1: tf_index_1,
        2: tf_index_2
    }[index_set]

####
#### Preprocessed query based on the two configs

# This function preprocesses the text given the index set, according to the specified config
def preprocess_query(text, index_set):
    assert index_set in {1, 2}
    if index_set == 1:
        return process_text(text, **config_1)
    elif index_set == 2:
        return process_text(text, **config_2)

#### 

In [21]:
##### Function check

assert isinstance(tf_index_1, dict)

assert isinstance(tf_index_1['computer'], list)
print('sample tf index for computer:', tf_index_1['computer'][:10])

assert isinstance(tf_index_1['examples'], list)
print('sample tf index for examples:', tf_index_1['examples'][:10])
#### 

sample tf index for computer: [[('4', 1), ('7', 1), ('10', 1), ('13', 1), ('19', 1), ('22', 1), ('23', 1), ('37', 1), ('40', 3), ('41', 1), ('44', 1), ('48', 1), ('57', 1), ('58', 2), ('63', 2), ('68', 1), ('71', 3), ('80', 1), ('82', 1), ('90', 1), ('92', 1), ('93', 3), ('95', 1), ('96', 2), ('104', 1), ('106', 2), ('123', 1), ('124', 1), ('143', 1), ('144', 1), ('146', 1), ('173', 1), ('175', 1), ('185', 1), ('188', 1), ('192', 1), ('202', 1), ('217', 1), ('218', 1), ('236', 2), ('237', 1), ('242', 1), ('251', 1), ('252', 2), ('254', 1), ('274', 1), ('276', 1), ('278', 3), ('282', 1), ('300', 1), ('303', 1), ('320', 1), ('322', 4), ('323', 1), ('331', 1), ('361', 1), ('409', 1), ('417', 5), ('436', 1), ('439', 1), ('461', 1), ('462', 1), ('464', 2), ('482', 1), ('488', 1), ('489', 1), ('491', 1), ('495', 2), ('496', 1), ('525', 1), ('530', 1), ('531', 1), ('557', 1), ('558', 2), ('561', 1), ('585', 1), ('595', 2), ('605', 1), ('617', 1), ('619', 1), ('644', 2), ('671', 1), ('675', 1)

In [22]:
##### Function check

assert isinstance(tf_index_2, dict)

assert isinstance(tf_index_2['computer'], list)
print('sample tf index for computer:', tf_index_1['computer'][:10])

assert isinstance(tf_index_2['examples'], list)
print('sample tf index for examples:', tf_index_2['examples'][:10])
#### 

sample tf index for computer: [[('4', 1), ('7', 1), ('10', 1), ('13', 1), ('19', 1), ('22', 1), ('23', 1), ('37', 1), ('40', 3), ('41', 1), ('44', 1), ('48', 1), ('57', 1), ('58', 2), ('63', 2), ('68', 1), ('71', 3), ('80', 1), ('82', 1), ('90', 1), ('92', 1), ('93', 3), ('95', 1), ('96', 2), ('104', 1), ('106', 2), ('123', 1), ('124', 1), ('143', 1), ('144', 1), ('146', 1), ('173', 1), ('175', 1), ('185', 1), ('188', 1), ('192', 1), ('202', 1), ('217', 1), ('218', 1), ('236', 2), ('237', 1), ('242', 1), ('251', 1), ('252', 2), ('254', 1), ('274', 1), ('276', 1), ('278', 3), ('282', 1), ('300', 1), ('303', 1), ('320', 1), ('322', 4), ('323', 1), ('331', 1), ('361', 1), ('409', 1), ('417', 5), ('436', 1), ('439', 1), ('461', 1), ('462', 1), ('464', 2), ('482', 1), ('488', 1), ('489', 1), ('491', 1), ('495', 2), ('496', 1), ('525', 1), ('530', 1), ('531', 1), ('557', 1), ('558', 2), ('561', 1), ('585', 1), ('595', 2), ('605', 1), ('617', 1), ('619', 1), ('644', 2), ('671', 1), ('675', 1)


---
## Section 3: Ranking  (80 points) <a class="anchor" id="ranking"></a>

[Back to Part 1](#part1)

Now that we have cleaned and processed our dataset, we can start building simple IR systems. 

For now, we consider *simple* IR systems, which involve computing scores from the tokens present in the document/query. More advanced methods are covered in later assignments.

We will implement the following methods in this section:
- [Section 3.1: Bag of Words](#bow) (10 points)
- [Section 3.2: TF-IDF](#tfidf) (15 points)
- [Section 3.3: Query Likelihood Model](#qlm) (35 points)
- [Section 3.4: BM25](#bm25) (20 points)

*All search functions should be able to handle multiple words queries.*

**Scoring policy:**
Your implementations in this section are scored based on the expected performance of your ranking functions.
You will get a full mark if your implementation meets the expected performance (measured by some evaluation metric).
Otherwise, you may get partial credit.
For example, if your *Bag of words* ranking function has 60% of expected performance, you will get 6 out of 10.

--- 

### Section 3.1: Bag of Words (10 points)<a class="anchor" id="bow"></a>

Probably the simplest IR model is the Bag of Words (BOW) model.
Implement a function that scores and ranks all the documents against a query using this model.   

- For consistency, you should use the count of the token and **not** the binary indicator.
- Use `float` type for the scores (even though the scores are integers in this case).
- No normalization of the scores is necessary, as the ordering is what we are interested in.
- If two documents have the same score, they can have any ordering: you are not required to disambiguate.


In [23]:
# TODO: Implement this! (10 points)
def bow_search(query, index_set):
    """
        Perform a search over all documents with the given query. 
        Note: You have to use the `get_index` function created in the previous cells
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query. 
    """
    index = get_index(index_set)
    processed_query = preprocess_query(query, index_set)
        
    # YOUR CODE HERE

    # one list with all the tuples
    total = [index[word] for word in processed_query]
    all_tuples = [tup for word in total for tuples in word for tup in tuples]
    
    score = dict()
    
    for tup in all_tuples:
        if tup[0] not in score:
            # {doc_id: score} 
            score[tup[0]] = float(tup[1])
        else:
            # update the score
            score[tup[0]] += float(tup[1])

    return sorted(score.items(), key=lambda s: s[1], reverse=True)


In [24]:
#### Function check

test_bow = bow_search("how to implement bag of words search", index_set=1)[:5]
assert isinstance(test_bow, list)
assert len(test_bow[0]) == 2
assert isinstance(test_bow[0][0], str)
assert isinstance(test_bow[0][1], float)

#### 

In [25]:

docs_by_id = dict(docs)
def print_results(docs, len_limit=50):    
    for i, (doc_id, score) in enumerate(docs):
        doc_content = docs_by_id[doc_id].strip().replace("\n", "\\n")[:len_limit] + "..."
        print(f"Rank {i}({score:.2}): {doc_content}")

test_bow_2 = bow_search("computer search word", index_set=2)[:5]
print(f"BOW Results:")
print_results(test_bow_2)


BOW Results:
Rank 0(1.3e+01): On Computing The Fast Fourier Transform\nCooley an...
Rank 1(1.2e+01): Variable Length Tree Structures Having Minimum Ave...
Rank 2(1.1e+01): A Modular Computer Sharing System\nAn alternative ...
Rank 3(1e+01): PEEKABIT, Computer Offspring of Punched\nCard PEEK...
Rank 4(9.0): Computer Simulation-Discussion of the\nTechnique a...


In [26]:

test_bow_1 = bow_search("computer search word", index_set=1)[:5]
print(f"BOW Results:")
print_results(test_bow_1)


BOW Results:
Rank 0(9.0): CURRICULUM 68 -- Recommendations for Academic\nPro...
Rank 1(9.0): Variable Length Tree Structures Having Minimum Ave...
Rank 2(7.0): Computer Formulation of the Equations of Motion Us...
Rank 3(7.0): The Effects of Multiplexing on a Computer-Communic...
Rank 4(6.0): Optimizing Bit-time Computer Simulation\nA major c...


In [27]:
print('top-5 docs for index1:', list(zip(*test_bow_1[:5]))[0])
print('top-5 docs for index2:', list(zip(*test_bow_2[:5]))[0])


top-5 docs for index1: ('1771', '1936', '1543', '2535', '678')
top-5 docs for index2: ('1525', '1936', '1844', '1700', '1366')



---

### Section 3.2: TF-IDF (15 points) <a class="anchor" id="tfidf"></a>

Before we implement the tf-idf scoring functions, let's first write a function to compute the document frequencies of all words.  

#### 3.2.1 Document frequency (5 points)
Compute the document frequencies of all tokens in the collection. 
Your code should return a dictionary with tokens as its keys and the number of documents containing the token as values.
For consistency, the values should have `int` type.

In [28]:
# TODO: Implement this! (5 points)
def compute_df(documents):
    """
        Compute the document frequency of all terms in the vocabulary
        Input: A list of documents
        Output: A dictionary with {token: document frequency (int)}
    """
    # YOUR CODE HERE
    
    df = defaultdict(int)
    
    for document in documents:
        # deal with duplicates
        for word in set(document):
            if word not in df:
                df[word] = 1
            else:
                df[word] += 1
                
    return df


In [29]:
#### Compute df based on the two configs

# get the document frequencies of each document
df_1 = compute_df([d[1] for d in doc_repr_1])
df_2 = compute_df([d[1] for d in doc_repr_2])

def get_df(index_set):
    assert index_set in {1, 2}
    return {
        1: df_1,
        2: df_2
    }[index_set]
####

In [30]:
#### Function check

print(df_1['computer'])
print(df_2['computer'])
####

597
11


---
#### 3.2.2 TF-IDF search (10 points)
Next, implement a function that computes a tf-idf score, given a query.
Use the following formulas for TF and IDF:

$$ TF=\log (1 + f_{d,t}) $$

$$ IDF=\log (\frac{N}{n_t})$$

where $f_{d,t}$ is the frequency of token $t$ in document $d$, $N$ is the number of total documents and $n_t$ is the number of documents containing token $t$.

**Note:** your implementation will be auto-graded assuming you have used the above formulas.


In [31]:
# TODO: Implement this! (10 points)
def tfidf_search(query, index_set):
    """
        Perform a search over all documents with the given query using tf-idf. 
        Note #1: You have to use the `get_index` (and the `get_df`) function created in the previous cells
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    index = get_index(index_set)
    df = get_df(index_set)
    processed_query = preprocess_query(query, index_set)
    
    N = len(docs)
    
    # YOUR CODE HERE
    
    def tf_idf(tf, df, N):
        """
            Returns a list with doc_ids and tf-idfs per word as a tuple
        """
        try:
            tf_idf = [(tup[0], np.log(1 + tup[1]) * np.log(N/df)) for tup in tf[0]]
        except:
            # the function cannot handle empty lists
            tf_idf = []

        return tf_idf
 
    # a list with lists containing tuples (doc_id, tf_idf)
    score_temp = [tf_idf(index[word], df[word], N) for word in processed_query]
  
    # ensure that the final list is N long
    score = {doc[0]:float(0) for doc in docs}
    
    for tup_list in score_temp:
        for tup in tup_list:
            if tup[0] in score:
                # this is basically the sum part for the score
                score[tup[0]] += tup[1]
    
    return sorted(score.items(), key=lambda s: s[1], reverse=True)

In [32]:

#### Function check
test_tfidf = tfidf_search("how to implement tf idf search", index_set=1)[:5]
assert isinstance(test_tfidf, list)
assert len(test_tfidf[0]) == 2
assert isinstance(test_tfidf[0][0], str)
assert isinstance(test_tfidf[0][1], float)

####

In [33]:

test_tfidf_2 = tfidf_search("computer word search", index_set=2)[:5]
print(f"TFIDF Results:")
print_results(test_tfidf_2)


TFIDF Results:
Rank 0(1.3e+01): PEEKABIT, Computer Offspring of Punched\nCard PEEK...
Rank 1(9.8): Variable Length Tree Structures Having Minimum Ave...
Rank 2(8.2): A Stochastic Approach to the Grammatical Coding of...
Rank 3(8.1): Full Table Quadratic Searching for Scatter Storage...
Rank 4(7.6): Use of Tree Structures for Processing Files\nIn da...


In [34]:

test_tfidf_1 = tfidf_search("computer word search", index_set=1)[:5]
print(f"TFIDF Results:")
print_results(test_tfidf_1)


TFIDF Results:
Rank 0(9.4): Variable Length Tree Structures Having Minimum Ave...
Rank 1(7.4): On the Feasibility of Voice Input to\nan On-line C...
Rank 2(7.3): Median Split Trees: A Fast Lookup Technique for Fr...
Rank 3(7.0): Execution Time Requirements for Encipherment Progr...
Rank 4(7.0): Storage and Search Properties of a Tree-Organized ...


In [35]:

print('top-5 docs for index1 with BOW search:', list(zip(*test_bow_1[:5]))[0])
print('top-5 docs for index2 with BOW search:', list(zip(*test_bow_2[:5]))[0])
print('top-5 docs for index1 with TF-IDF search:', list(zip(*test_tfidf_1[:5]))[0])
print('top-5 docs for index2 with TF-IDF search:', list(zip(*test_tfidf_2[:5]))[0])



top-5 docs for index1 with BOW search: ('1771', '1936', '1543', '2535', '678')
top-5 docs for index2 with BOW search: ('1525', '1936', '1844', '1700', '1366')
top-5 docs for index1 with TF-IDF search: ('1936', '2054', '3041', '2620', '944')
top-5 docs for index2 with TF-IDF search: ('1700', '1936', '1235', '2018', '849')


--- 

### Section 3.3: Query Likelihood Model (35 points) <a class="anchor" id="qlm"></a>

In this section, you will implement a simple query likelihood model. 


#### 3.3.1 Naive QL (15 points)

First, let us implement a naive version of a QL model, assuming a multinomial unigram language model (with a uniform prior over the documents). 



In [36]:
#### Document length for normalization

def doc_lengths(documents):
    doc_lengths = {doc_id:len(doc) for (doc_id, doc) in documents}
    return doc_lengths

doc_lengths_1 = doc_lengths(doc_repr_1)
doc_lengths_2 = doc_lengths(doc_repr_2)

def get_doc_lengths(index_set):
    assert index_set in {1, 2}
    return {
        1: doc_lengths_1,
        2: doc_lengths_2
    }[index_set]
####

In [381]:
# TODO: Implement this! (15 points)
def naive_ql_search(query, index_set):
    """
        Perform a search over all documents with the given query using a naive QL model. 
        Note #1: You have to use the `get_index` (and get_doc_lengths) function created in the previous cells
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    index = get_index(index_set)
    doc_lengths = get_doc_lengths(index_set)
    processed_query = preprocess_query(query, index_set)
    
    # YOUR CODE HERE
    
    # this includes relevant and irrelevant doc_ids --> only the relevant doc_ids will be updated
    score = {doc_id: 0 for doc_id in doc_lengths.keys()}
    
    for word in processed_query:  
        # list with tuples (doc_id, tf) 
        tf_list = index[word]
        
        # skip all the words that are not present in index
        if len(tf_list) != 0:           

            # retrieve doc_id, tf, and dl
            for doc_id, tf in tf_list[0]:
                dl = doc_lengths[doc_id]

                if score[doc_id] == 0:
                    # the score can be updated with *=
                    score[doc_id] = 1

                # update the scores
                score[doc_id] *= tf / dl  
                
    return sorted(score.items(), key=lambda s: s[1], reverse=True)


In [262]:
#### Function check
test_naiveql = naive_ql_search("report", index_set=1)[:5]
print(f"Naive QL Results:")
print_results(test_naiveql)
####

Naive QL Results:
Rank 0(0.2): A Report Writer For COBOL...
Rank 1(0.2): A CRT Report Generating System...
Rank 2(0.17): Preliminary Report-International Algebraic Languag...
Rank 3(0.17): Supplement to the ALGOL 60 Report...
Rank 4(0.14): ALGOL Sub-Committee Report - Extensions...


In [263]:
#### Please do not change this. This cell is used for grading.

In [264]:
#### Please do not change this. This cell is used for grading.

In [265]:
#### Please do not change this. This cell is used for grading.

In [266]:
#### Please do not change this. This cell is used for grading.

---
#### 3.3.2 QL (20 points)
Now, let's implement a QL model that handles the issues with the naive version. In particular, you will implement a QL model with Jelinek-Mercer Smoothing. That means an interpolated score is computed per word - one term is the same as the previous naive version, and the second term comes from a unigram language model. In addition, you should accumulate the scores by summing the **log** (smoothed) probability which leads to better numerical stability.

In [229]:
# TODO: Implement this! (20 points)

# YOUR CODE HERE
# raise NotImplementedError()

def ql_search(query, index_set):
    """
        Perform a search over all documents with the given query using a QL model 
        with Jelinek-Mercer Smoothing (set smoothing=0.1). 
        
        
        Note #1: You have to use the `get_index` (and get_doc_lengths) function created in the previous cells
        Note #2: You might have to create some variables beforehand and use them in this function
        
        
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    index = get_index(index_set)
    doc_lengths = get_doc_lengths(index_set)
    processed_query = preprocess_query(query, index_set)
    
    # YOUR CODE HERE
    
    smoothing = 0.1
    # collection length
    cl = len(doc_lengths)
    
    score = {doc_id: float(0) for doc_id in doc_lengths.keys()}
    
    for word in processed_query:

        if index[word] != []:
            cf = np.sum(list(dict(index[word][0]).values()))

            # list with tuples (doc_id, tf)
            tf_list = index[word]

            # retrieve doc_id, tf, and dl
            for doc_id, tf in tf_list[0]:
                dl = doc_lengths[doc_id]

                # update the scores; first entry is naive
                if score[doc_id] == 0:
                    score[doc_id] = tf / dl
                # this handles a query; second entry + is smoothed 
                else:
                    # there are two ways to implement this according to piazza
                    
                    # method 1 from the slides which passes the test
#                     score[doc_id] += abs(np.log((smoothing * (tf / dl)) + ((1 - smoothing) * (cf / cl))))
                    
                    # recommended method according to piazza which does not pass the test
                    score[doc_id] += abs(np.log(((1 - smoothing) * (tf / dl)) + (smoothing * (cf / cl))))
                    
        
    return sorted(score.items(), key=lambda s: s[1], reverse=True)


In [230]:
#### Function check
test_ql_results = ql_search("report", index_set=1)[:5]
print_results(test_ql_results)
print()
test_ql_results_long = ql_search("report " * 10, index_set=1)[:5]
print_results(test_ql_results_long)
####

Rank 0(0.2): A Report Writer For COBOL...
Rank 1(0.2): A CRT Report Generating System...
Rank 2(0.17): Preliminary Report-International Algebraic Languag...
Rank 3(0.17): Supplement to the ALGOL 60 Report...
Rank 4(0.14): ALGOL Sub-Committee Report - Extensions...

Rank 0(4.5e+01): The State of Computer Oriented Curricula in Busine...
Rank 1(4.5e+01): A Profile of the Programmer\nSynopsis: 549 members...
Rank 2(4.5e+01): A Relational Model of Data for Large Shared Data B...
Rank 3(4.5e+01): Coding Clinical Laboratory Data For Automatic Stor...
Rank 4(4.4e+01): Studies in Machine Cognition Using The Game of Pok...


In [231]:
#### Please do not change this. This cell is used for grading.

In [232]:
#### Please do not change this. This cell is used for grading.

In [233]:
#### Please do not change this. This cell is used for grading.

In [234]:
#### Please do not change this. This cell is used for grading.

--- 

### Section 3.4: BM25 (20 points) <a class="anchor" id="bm25"></a>

In this section, we will implement the BM25 scoring function. 


In [235]:
# TODO: Implement this! (20 points)
def bm25_search(query, index_set):
    """
        Perform a search over all documents with the given query using BM25. Use k_1 = 1.5 and b = 0.75
        Note #1: You have to use the `get_index` (and `get_doc_lengths`) function created in the previous cells
        Note #2: You might have to create some variables beforehand and use them in this function
        
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    
    index = get_index(index_set)
    df = get_df(index_set)
    doc_lengths = get_doc_lengths(index_set)
    processed_query = preprocess_query(query, index_set)
    
    # YOUR CODE HERE
    
    score = {doc_id: float(0) for doc_id in doc_lengths.keys()}
    
    k_1 = 1.5
    b = 0.75
    
    for word in processed_query:
        tf_list = index[word]
        
        if tf_list != []:
        
            for doc_id, tf in tf_list[0]:
    
                # split formula
                p1 = np.log(len(doc_lengths) / df[word])
                p2 = (k_1 + 1) * tf
                p3 = k_1 * ((1 - b) + b * (doc_lengths[doc_id] / np.mean(list(doc_lengths.values())))) + tf
    
                # construct bm25
                bm25 = p1 * (p2 / p3)

                if score[doc_id] == 0:
                    score[doc_id] = bm25 
                else:
                    score[doc_id] += bm25 
    
    return sorted(score.items(), key=lambda s: s[1], reverse=True)


In [236]:
#### Function check
test_bm25_results = bm25_search("report", index_set=1)[:5]
print_results(test_bm25_results)
####

Rank 0(6.7): A Report Writer For COBOL...
Rank 1(6.7): A CRT Report Generating System...
Rank 2(6.6): Preliminary Report-International Algebraic Languag...
Rank 3(6.6): Supplement to the ALGOL 60 Report...
Rank 4(6.5): ALGOL Sub-Committee Report - Extensions...


In [237]:
#### Please do not change this. This cell is used for grading.

In [238]:
#### Please do not change this. This cell is used for grading.

In [239]:
#### Please do not change this. This cell is used for grading.

In [240]:
#### Please do not change this. This cell is used for grading.


---

### 3.5. Test Your Functions

The widget below allows you to play with the search functions you've written so far. Use this to test your search functions and ensure that they work as expected.

In [241]:
#### Highlighter function
# class for results
ResultRow = namedtuple("ResultRow", ["doc_id", "snippet", "score"])
# doc_id -> doc
docs_by_id = dict((d[0], d[1]) for d in docs)

def highlight_text(document, query, tol=17):
    import re
    tokens = tokenize(query)
    regex = "|".join(f"(\\b{t}\\b)" for t in tokens)
    regex = re.compile(regex, flags=re.IGNORECASE)
    output = ""
    i = 0
    for m in regex.finditer(document):
        start_idx = max(0, m.start() - tol)
        end_idx = min(len(document), m.end() + tol)
        output += "".join(["...",
                        document[start_idx:m.start()],
                        "<strong>",
                        document[m.start():m.end()],
                        "</strong>",
                        document[m.end():end_idx],
                        "..."])
    return output.replace("\n", " ")


def make_results(query, search_fn, index_set):
    results = []
    for doc_id, score in search_fn(query, index_set):
        highlight = highlight_text(docs_by_id[doc_id], query)
        if len(highlight.strip()) == 0:
            highlight = docs_by_id[doc_id]
        results.append(ResultRow(doc_id, highlight, score))
    return results
####

In [242]:
# TODO: Set this to the function you want to test!
# this function should take in a query (string)
# and return a sorted list of (doc_id, score) 
# with the most relevant document in the first position
search_fn = bm25_search
index_set = 1

text = widgets.Text(description="Search Bar", width=200)
display(text)

def handle_submit(sender):
    print(f"Searching for: '{sender.value}'")
    
    results = make_results(sender.value, search_fn, index_set)
    
    # display only the top 5
    results = results[:5]
    
    body = ""
    for idx, r in enumerate(results):
        body += f"<li>Document #{r.doc_id}({r.score}): {r.snippet}</li>"
    display(HTML(f"<ul>{body}</ul>"))
    

text.on_submit(handle_submit)

Text(value='', description='Search Bar')

---

## Section 4: Evaluation (40 points) <a class="anchor" id="evaluation"></a>

[Back to Part 1](#part1)

In order to analyze the effectiveness of retrieval algorithms, we first have to learn how to evaluate such a system. In particular, we will work with offline evaluation metrics. These metrics are computed on a dataset with known relevance judgements.

Implement the following evaluation metrics. 

1. Precision (7 points)
2. Recall (7 points)
3. Mean Average Precision (13 points)
4. Expected Reciprocal Rank (13 points)

---
### 4.1 Read relevance labels

Let's take a look at the `qrels.text` file, which contains the ground truth relevance scores. The relevance labels for CACM are binary - either 0 or 1. 


In [243]:
!head ./datasets/qrels.text

01 1410  0 0
01 1572  0 0
01 1605  0 0
01 2020  0 0
01 2358  0 0
02 2434  0 0
02 2863  0 0
02 3078  0 0
03 1134  0 0
03 1613  0 0


---

The first column is the query_id and the second column is the document_id. We can safely ignore the 3rd and 4th columns.

In [244]:
def read_qrels(root_folder = "./datasets/"):
    """
        Reads the qrels.text file. 
        Output: A dictionary: query_id -> [list of relevant documents]
    """
    with open(os.path.join(root_folder, "qrels.text")) as reader:
        lines = reader.readlines()
    
    from collections import defaultdict
    relevant_docs = defaultdict(set)
    for line in lines:
        query_id, doc_id, _, _ = line.split()
        relevant_docs[str(int(query_id))].add(doc_id)
    return relevant_docs


In [245]:
#### Function check
qrels = read_qrels()

assert len(qrels) == 52, "There should be 52 queries with relevance judgements"
assert sum(len(j) for j in qrels.values()) == 796, "There should be a total of 796 Relevance Judgements"

assert np.min(np.array([len(j) for j in qrels.values()])) == 1
assert np.max(np.array([len(j) for j in qrels.values()])) == 51

####

---
**Note:** For a given query `query_id`, you can assume that documents *not* in `qrels[query_id]` are not relevant to `query_id`. 


---
### 4.2 Precision (7 points)
Implement the `precision@k` metric:

In [246]:
# TODO: Implement this! (7 points)
def precision_k(results, relevant_docs, k):
    """
        Compute Precision@K
        Input: 
            results: A sorted list of 2-tuples (document_id, score), 
                    with the most relevant document in the first position
            relevant_docs: A set of relevant documents. 
            k: the cut-off
        Output: Precision@K
    """
    if k > len(results):
        k = len(results)
        
    # YOUR CODE HERE
    
    # results till k
    r_k = dict(results[:k]).keys()
        
    # intersection
    return len(set(r_k) & set(relevant_docs)) / k


In [247]:

#### Function check
qid = queries[0][0]
qtext = queries[0][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
precision = precision_k(results, qrels[qid], 10)
print(f'precision@10 = {precision}')
####

query: What articles exist which deal with TSS (Time Sharing System), an
operating system for IBM computers?
precision@10 = 0.2


---
### 4.3 Recall (7 points)
Implement the `recall@k` metric:

In [248]:
# TODO: Implement this! (7 points)
def recall_k(results, relevant_docs, k):
    """
        Compute Recall@K
        Input: 
            results: A sorted list of 2-tuples (document_id, score), with the most relevant document in the first position
            relevant_docs: A set of relevant documents. 
            k: the cut-off
        Output: Recall@K
    """
    # YOUR CODE HERE
    
    # results till k
    r_k = set(dict(results[:k]).keys())
    
    return len(r_k & set(relevant_docs)) / len(relevant_docs)


In [249]:
#### Function check
qid = queries[10][0]
qtext = queries[10][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
recall = recall_k(results, qrels[qid], 10)
print(f'recall@10 = {recall}')
####

query: SETL, Very High Level Languages
recall@10 = 0.3157894736842105


---
### 4.4 Mean Average Precision (13 points)
Implement the `map` metric:

In [250]:
# TODO: Implement this! (12 points)
def average_precision(results, relevant_docs):
    """
        Compute Average Precision (for a single query - the results are 
        averaged across queries to get MAP in the next few cells)
        Hint: You can use the recall_k and precision_k functions here!
        Input: 
            results: A sorted list of 2-tuples (document_id, score), with the most 
                    relevant document in the first position
            relevant_docs: A set of relevant documents. 
        Output: Average Precision
    """
    # YOUR CODE HERE
    
    # sum(precision at rank k for document) / len(relevant_docs)
    
    numerator = 0
    
    for index, tup in enumerate(results, 1):
        if tup[0] in relevant_docs:
            numerator += precision_k(results, relevant_docs, index)
    
    return numerator / len(relevant_docs)


In [251]:
#### Function check
qid = queries[20][0]
qtext = queries[20][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
mean_ap = average_precision(results, qrels[qid])
print(f'MAP = {mean_ap}')
####

query: computational complexity, intractability, class-complete reductions,
algorithms and efficiency
MAP = 0.17240404110559454


---
### 4.5 Expected Reciprocal Rank (13 points)
Implement the `err` metric:

In [252]:
# TODO: Implement this! (12 points)
def err(results, relevant_docs):
    """
        Compute the expected reciprocal rank.
        Hint: https://dl.acm.org/doi/pdf/10.1145/1645953.1646033?download=true
        Input: 
            results: A sorted list of 2-tuples (document_id, score), with the most 
                    relevant document in the first position
            relevant_docs: A set of relevant documents. 
        Output: ERR
        
    """
    # YOUR CODE HERE
    
    ERR = 0
    p = 1
    
    for rank in range(len(results)):
        if results[rank][0] in relevant_docs:
            g = 1
        else:
            g = 0

        # g consists of {0, 1} because it's binary ; g_max is always 1
        R = ((2**g) - 1) / (2**1)    
    
        # can't divide by 0
        ERR = ERR + (p * (R / (rank + 1)))
#         ERR += p * (R / (rank + 1))
        p = p * (1 - R)  
    
    return ERR

In [253]:
#### Function check
qid = queries[30][0]
qtext = queries[30][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
ERR = err(results, qrels[qid])
print(f'ERR = {ERR}')
####

query: I'd like to find articles describing the use of singular value decomposition
in digital image processing.  Applications include finding approximations
to the original image and restoring images that are subject to noise. An
article on the subject is H.C. Andrews and C.L. Patterson "Outer product
expansions and their uses in digital image processing", American Mathematical
Monthly, vol. 82.
ERR = 0.625


---
### 4.6 Evaluate Search Functions

Let's define some metrics@k using [partial functions](https://docs.python.org/3/library/functools.html#functools.partial)

In [254]:
#### metrics@k functions

recall_at_1 = partial(recall_k, k=1)
recall_at_5 = partial(recall_k, k=5)
recall_at_10 = partial(recall_k, k=10)
precision_at_1 = partial(precision_k, k=1)
precision_at_5 = partial(precision_k, k=5)
precision_at_10 = partial(precision_k, k=10)


list_of_metrics = [
    ("ERR", err),
    ("MAP", average_precision),
    ("Recall@1",recall_at_1),
    ("Recall@5", recall_at_5),
    ("Recall@10", recall_at_10),
    ("Precision@1", precision_at_1),
    ("Precision@5", precision_at_5),
    ("Precision@10", precision_at_10)]
####

---

The following function evaluates a `search_fn` using the `metric_fn`. Note that the final number is averaged over all the queries

In [255]:
recall_at_1 = partial(recall_k, k=1)
recall_at_5 = partial(recall_k, k=5)
recall_at_10 = partial(recall_k, k=10)
recall_at_20 = partial(recall_k, k=20)
precision_at_1 = partial(precision_k, k=1)
precision_at_5 = partial(precision_k, k=5)
precision_at_10 = partial(precision_k, k=10)
precision_at_20 = partial(precision_k, k=20)

list_of_metrics = [
    ("ERR", err),
    ("MAP", average_precision),
    ("Recall@1",recall_at_1),
    ("Recall@5", recall_at_5),
    ("Recall@10", recall_at_10),
    ("Recall@20", recall_at_20),
    ("Precision@1", precision_at_1),
    ("Precision@5", precision_at_5),
    ("Precision@10", precision_at_10),
    ("Precision@20", precision_at_20)]

#### Evaluate a search function

list_of_search_fns = [
    ("BOW", bow_search),
    ("TF-IDF", tfidf_search),
    ("NaiveQL", naive_ql_search),
    ("QL", ql_search),
    ("BM25", bm25_search)
]

def evaluate_search_fn(search_fn, metric_fns, index_set=None):
    # build a dict query_id -> query 
    queries_by_id = dict((q[0], q[1]) for q in queries)
    
    metrics = {}
    for metric, metric_fn in metric_fns:
        metrics[metric] = np.zeros(len(qrels), dtype=np.float32)
    
    for i, (query_id, relevant_docs) in enumerate(qrels.items()):
        query = queries_by_id[query_id]
        if index_set:
            results = search_fn(query, index_set)
        else:
            results = search_fn(query)
        
        for metric, metric_fn in metric_fns:
            metrics[metric][i] = metric_fn(results, relevant_docs)
    
    final_dict = {}
    for metric, metric_vals in metrics.items():
        final_dict[metric] = metric_vals.mean()
    
    return final_dict

####

sample_results = [(i,None) for i in [4,0,1,17,2,5,6,13,9,10,8,20,3,15,16]]
sample_qrel = [1,8,5,13,15,2]
assert precision_k(sample_results, sample_qrel, 1) == 0.
assert precision_k(sample_results, sample_qrel, 5) == 0.4
assert precision_k(sample_results, sample_qrel, 10) == 0.4
assert precision_k(sample_results, sample_qrel, 20) == 0.4
assert recall_k(sample_results, sample_qrel, 1) == 0.
assert np.allclose(recall_k(sample_results, sample_qrel, 5), 1./3.)
assert np.allclose(recall_k(sample_results, sample_qrel, 10), 2./3.)
assert recall_k(sample_results, sample_qrel, 20) == 1.
assert np.allclose(average_precision(sample_results, sample_qrel), 0.436, 0.001)
assert np.allclose(err(sample_results, sample_qrel), 0.2492, 0.001)


for tup in list_of_search_fns:
    print(tup[0])
    print(evaluate_search_fn(tup[1], list_of_metrics, index_set=2))

BOW
{'ERR': 0.11996831, 'MAP': 0.06623642, 'Recall@1': 0.009497121, 'Recall@5': 0.04654979, 'Recall@10': 0.080543, 'Recall@20': 0.12496212, 'Precision@1': 0.07692308, 'Precision@5': 0.080769226, 'Precision@10': 0.07307692, 'Precision@20': 0.058653846}
TF-IDF
{'ERR': 0.41484997, 'MAP': 0.2551034, 'Recall@1': 0.08998284, 'Recall@5': 0.21299928, 'Recall@10': 0.2675624, 'Recall@20': 0.35488722, 'Precision@1': 0.53846157, 'Precision@5': 0.33846155, 'Precision@10': 0.25, 'Precision@20': 0.19038464}
NaiveQL
{'ERR': 0.004748977, 'MAP': 0.0055716513, 'Recall@1': 0.0, 'Recall@5': 0.0, 'Recall@10': 0.0, 'Recall@20': 0.0, 'Precision@1': 0.0, 'Precision@5': 0.0, 'Precision@10': 0.0, 'Precision@20': 0.0}
QL
{'ERR': 0.2787004, 'MAP': 0.14574434, 'Recall@1': 0.045611043, 'Recall@5': 0.10767846, 'Recall@10': 0.15969984, 'Recall@20': 0.22813624, 'Precision@1': 0.32692307, 'Precision@5': 0.21153846, 'Precision@10': 0.16923077, 'Precision@20': 0.13365385}
BM25
{'ERR': 0.4261782, 'MAP': 0.30067486, 'Recall

## Section 5: Analysis (30 points) <a class="anchor" id="analysis"></a>

[Back to Part 1](#part1)

In the final section of Part1, we will compare the different term-based IR algorithms and different preprocessing configurations and analyze their advantages and disadvantages.

### Section 5.1: Plot (20 points)

First, gather the results. The results should consider the index set, the different search functions and different metrics. Plot the results in bar charts, per metric, with clear labels.

**Rubric:**
- Each Metric is plotted: 7 points
- Each Method is plotted: 7 points
- Clear titles, x label, y labels and legends (if applicable): 6 points

In [145]:
# YOUR CODE HERE

# the order makes the plot more clear
list_of_metrics = [
    ("ERR", err),
    ("MAP", average_precision),
    ("Recall@1",recall_at_1),
    ("Precision@1", precision_at_1),
    ("Recall@5", recall_at_5),
    ("Precision@5", precision_at_5),
    ("Recall@10", recall_at_10),
    ("Precision@10", precision_at_10),
    ("Recall@20", recall_at_20),
    ("Precision@20", precision_at_20)]

# this is the final dict for the plots
complete = dict() 
# temporary dict to differentiate between index 1 and 2
temp = dict()   

# for both index sets
for x in [1,2]:
    for search in list_of_search_fns:
        temp[search[0]] = evaluate_search_fn(search[1], list_of_metrics, x)
    complete[x] = temp
    # reset dict
    temp = dict()

methods = [search[0] for search in list_of_search_fns]
metrics = [metric[0] for metric in list_of_metrics]
    
# split data according to index
data_1 = defaultdict(list)
data_2 = defaultdict(list)

# temporary x-axis labels which are numeric
temp_x = np.arange(len(methods)) # + 1

# add values to complete
for metric in metrics:

    # index 1
    values_1 = [complete[1][method][metric] for method in methods] 
    data_1[metric].append(values_1)
    
    # index 2
    values_2 = [complete[2][method][metric] for method in methods]
    data_2[metric].append(values_2)
    
# plot
fig, ax = plt.subplots(5, 2, figsize=(20, 20))

# adjust y and pad for spacing
fig.suptitle('Metric per method per index set', y=1.02, size='xx-large')
fig.tight_layout(pad=5.0)

# this is a lifesaver
ax = ax.flatten()

for index, metric in enumerate(metrics):
    width = 0.35

    # create plots for indices 1 and 2
    ax[index].bar(temp_x - width/2, data_1[metric][0], width, color='salmon', label='Index set 1')
    ax[index].bar(temp_x + width/2, data_2[metric][0], width, color='skyblue', label='Index set 2')

    # set to actual labels
    ax[index].set_xticks(temp_x)
    ax[index].set_xticklabels(methods)

    # set y-axis 
    ax[index].set_ylabel(metric + ' score')
    ax[index].set_title('Figure {}: Results of {}'.format(index + 1, metric))
    
    # set same ylim for all plot to make differences between plots more clear
    ax[index].set_ylim((0, 0.60))

    ax[index].legend()

   
fig.show()

KeyboardInterrupt: 

---
### Section 5.2: Summary (10 points)
Write a summary of what you observe in the results.
Your summary should compare results across the 2 indices and the methods being used. State what you expected to see in the results, followed by either supporting evidence *or* justify why the results did not support your expectations.      

#### Summary of 2 indices
Across the two indices in the figures plotted above, you can clearly see that index set 2 (represented by the blue bars) has higher scores in nearly all graphs. Only the QL in Figures 3 and 5 (Recall@1 and Recall@5) give a higher score for index set 1 than set 2, but the differences are quite small. These findings are in line with our expectations, as the pre-processing of index set 2 includes lowercasing, stemming, and the removal of stopwords, as opposed to index set 1 for which only lowercasing is used.

#### Summary across methods 
We expected that the BM25 method is the best performing metric overall. This expectation is supported by the plotted figures because in all the figures (i.e. all of the evaluation metrics used), BM25 performs best. We also expected the BOW to perform worse than the TF-IDF, as the BOW only takes the frequency of a term into account, while the TF-IDF also uses the number of documents that a term occurs in. This expectation is also supported by the Figures because the outcome of TF-IDF is higher in all of them.

Furthermore, we expected the recall score to increase with the rank and the precision to decrease because if the number of documents increases, the percentage of relevant items will decrease, whereas the number of retrieved relevant items will increase. This is also supported by the Figures above.

Moreover, we expected low scores for NaiveQL (i.e. when the tf = 0, the product will also be 0). This expectation is supported by the Figures. In addition, we expected the QL to perform better than the NaiveQL model because the QL model handles the issues with the naive version by using Jelinek-Mercer Smoothing. This expectation is also supported by the Figures plotted above as the QL model performs better in every Figure. 


---
---
# Part 2: Semantic-based Matching (85 points) <a class="anchor" id="part2"></a>

[Back to top](#top)

We will now experiment with methods that go beyond lexical methods like TF-IDF, which operate at the word level and are high dimensional and sparse, and look at methods which constructs low dimensional dense representations of queries and documents. 

Since these low-dimensional methods have a higher time complexity, they are typically used in conjunction with methods like BM-25. That is, instead of searching through potentially million documents to find matches using low dimensional vectors, a list of K documents are retrieved using BM25, and then **re-ranked** using the other method. This is the method that is going to be applied in the following exercises. 

LSI/LDA takes documents that are similar on a semantic level - for instance, if they are describing the same topic - and projects them into nearby vectors, despite having low lexical overlap.

In this assignment, you will use `gensim` to create LSI/LDA models and use them in re-ranking. 

**Note**: The following exercises only uses `doc_repr_2` and `config_2`

Table of contents:
- [Section 6: LSI](#lsi) (15 points)
- [Section 7: LDA](#lda) (10 points)
- [Section 8: Word2Vec/Doc2Vec](#2vec) (20 points)
- [Section 8: Re-ranking](#reranking) (10 points)
- [Section 9: Re-ranking Evaluation](#reranking_eval) (30 points)

---
## Section 6: Latent Semantic Indexing (LSI) (15 points) <a class="anchor" id="lsi"></a>

[Back to Part 2](#part2)

LSI is one of the methods to embed the queries and documents into vectors. It is based on a method similar to Principal Component Analysis (PCA) for obtaining a dense concept matrix out of the sparse term-document matrix.

See [wikipedia](https://en.wikipedia.org/wiki/Latent_semantic_analysis), particularly [#Mathematics_of_LSI](https://en.wikipedia.org/wiki/Latent_semantic_analysis#Mathematics_of_LSI).

In [71]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel, LsiModel, Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim import downloader as g_downloader
# gensim uses logging, so set it up 
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

---
### Section 6.1: Cosine Similarity (5 points)<a class="anchor" id="cosing_sim"></a>
Before we begin, let us first define our method of similarity for the LSI model, the cosine similarity:

$$\text{similarity} = \cos(\theta) = {\mathbf{A} \cdot \mathbf{B} \over \|\mathbf{A}\| \|\mathbf{B}\|} = \frac{ \sum\limits_{i=1}^{n}{A_i  B_i} }{ \sqrt{\sum\limits_{i=1}^{n}{A_i^2}}  \sqrt{\sum\limits_{i=1}^{n}{B_i^2}} }$$

Since we are using gensim, the types of vectors returned by their classes are of the form defined below (they are not just simple vectors):

In [72]:
# 1, 2, 3 are either latent dimensions (LSI), or topics (LDA)
# The second value in each tuple is a number (LSI) or a probability (LDA)  
example_vec_1 = [(1, 0.2), (2, 0.3), (3, 0.4)]
example_vec_2 = [(1, 0.2), (2, 0.7), (3, 0.4)]

---
**Implementation (2+3 points):**
Now, implement the `dot product` operation on these types of vectors and using this operator, implement the `cosine similarity` (don't forget: two functions to implement!):

In [73]:
# TODO: Implement this! (2 points)
def dot(vec_1,vec_2): 
    """
        vec_1 and vec_2 are of the form: [(int, float), (int, float), ...]
        Return the dot product of two such vectors, computed only on the floats
        You can assume that the lengths of the vectors are the same, and the dimensions are aligned 
            i.e you won't get: vec_1 = [(1, 0.2)] ; vec_2 = [(2, 0.3)] 
                                (dimensions are unaligned and lengths are different)
    """
    # YOUR CODE HERE
    
    values_1 = np.array([tup[1] for tup in vec_1])
    values_2 = np.array([tup[1] for tup in vec_2])
     
    return np.dot(values_1, values_2)

# TODO: Implement this! (3 points)
def cosine_sim(vec_1, vec_2):
    # YOUR CODE HERE
    
    if len(vec_1) == 0 or len(vec_2) == 0:
        return 0
    
    numerator = dot(vec_1, vec_2)
    denominator_a = np.sqrt(dot(vec_1, vec_1))
    denominator_b = np.sqrt(dot(vec_2, vec_2))
    
    return numerator / (denominator_a * denominator_b)

In [74]:
##### Function check
print(f'vectors: {(example_vec_1,example_vec_2)}')
print(f'dot product = {dot(example_vec_1,example_vec_2)}')
print(f'cosine similarity = {cosine_sim(example_vec_1,example_vec_2)}')
##### 

vectors: ([(1, 0.2), (2, 0.3), (3, 0.4)], [(1, 0.2), (2, 0.7), (3, 0.4)])
dot product = 0.41000000000000003
cosine similarity = 0.9165587597202866


In [75]:
#### Please do not change this. This cell is used for grading.

---
### Section 6.2: LSI Retrieval (10 points)<a class="anchor" id="lsi_retrieval"></a>
LSI retrieval is simply ranking the documents based on their cosine similarity to the query vector.
First, let's write a parent class for vector-based retrieval models:

In [76]:
class VectorSpaceRetrievalModel:
    """
        Parent class for Dense Vector Retrieval models
    """
    def __init__(self, doc_repr):
        """
            document_collection: 
                [
                    (doc_id_1, [token 1, token 2, ...]), 
                    (doc_id_2, [token 1, token 2, ....]) 
                    ...
                ]

        """
        self.doc_repr = doc_repr
        self.documents = [_[1] for _ in self.doc_repr]
        
        # construct a dictionary
        self.dictionary = Dictionary(self.documents)
        # Filter out words that occur less than 20 documents, or more than 50% of the documents.
        self.dictionary.filter_extremes(no_below=10)
        self.corpus = [self.dictionary.doc2bow(doc) for doc in self.documents]
    
        # Make a index to word dictionary.
        temp = self.dictionary[0]  # This is only to "load" the dictionary.
        self.id2word = self.dictionary.id2token
        
        # this is set by the train_model function
        self.model = None
        
        
    def vectorize_documents(self):
        """
            Returns a doc_id -> vector dictionary
        """
        vectors = {}
        for (doc_id, _), cc in zip(self.doc_repr, self.corpus):
            vectors[doc_id] = self.model[cc]
        return vectors

    def vectorize_query(self, query):
        # Note the use of config_2 here!
        query = process_text(query, **config_2)
        query_vector = self.dictionary.doc2bow(query)
        return self.model[query_vector]
    
    def train_model(self):
        """
            Trains a model and sets the 'self.model' variable. 
            Make sure to use the variables created in the __init__ method.
            e.g the variables which may be useful: {corpus, dictionary, id2word}
        """
        raise NotImplementedError()

---
**Implementation (5 points):**
Implement the `train_model` method in the following class (note that this is only one line of code in `gensim`!). Ensure that the parameters defined in the `__init__` method are not changed, and are *used in the `train_method` function*. Normally, the hyperaparameter space will be searched using grid search / other methods - in this assignment we have provided the hyperparameters for you.

The last two lines of code train an LSI model on the list of documents which have been stemmed, lower-cased and have stopwords removed. 

In [77]:
# TODO: Implement this! (5 points)
class LsiRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        self.num_topics = 100
        self.chunksize = 2000
    
    def train_model(self):
        # YOUR CODE HERE
        
        self.model = LsiModel(corpus=self.corpus, num_topics=self.num_topics, chunksize=self.chunksize, 
                              id2word=self.id2word)
        
        return self.model

In [78]:
##### Function check
lsi = LsiRetrievalModel(doc_repr_2)
lsi.train_model()

# you can now get an LSI vector for a given query in the following way:
lsi.vectorize_query("report")
##### 

2022-02-28 12:45:21,226 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-02-28 12:45:21,320 : INFO : built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115969 corpus positions)
2022-02-28 12:45:21,324 : INFO : discarding 4740 tokens: [('repeat', 8), ('glossari', 7), ('inspect', 8), ('uncol', 2), ('rung', 9), ('secant', 2), ('.', 1603), ('acceler', 6), ('diverg', 3), ('induc', 9)]...
2022-02-28 12:45:21,325 : INFO : keeping 1197 tokens which were in no less than 10 and no more than 1602 (=50.0%) documents
2022-02-28 12:45:21,326 : INFO : resulting dictionary: Dictionary(1197 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...)
2022-02-28 12:45:21,378 : INFO : using serial LSI version on this node
2022-02-28 12:45:21,379 : INFO : updating model with new documents
2022-02-28 12:45:21,379 : INFO : preparing a new chunk of documents
2022-02-28 12:45:21,389 : INFO : using 100 extra sam

[(0, 0.01521491725948722),
 (1, -0.01629877956744725),
 (2, -0.00022090020161401498),
 (3, -0.001864106555096402),
 (4, -0.009393325767764419),
 (5, -0.0047262323785759175),
 (6, 0.027135918609908008),
 (7, 0.016829652563226362),
 (8, -0.03170669323782037),
 (9, -0.0007639933448842992),
 (10, 0.001982153109922146),
 (11, -0.017518304174411553),
 (12, 0.0004788103665507547),
 (13, 0.0017353431621068335),
 (14, 0.0038270016678000358),
 (15, 0.005327139769397365),
 (16, 0.005473456301187418),
 (17, 0.001980295355525054),
 (18, -0.0180366998527497),
 (19, 0.020726092610355667),
 (20, -0.010961546096691203),
 (21, -0.014247118583679915),
 (22, 0.04649576112142203),
 (23, 0.025581816959192552),
 (24, -0.010651526017188107),
 (25, -0.009740688204733705),
 (26, 0.006286907244630003),
 (27, 0.07437705689579789),
 (28, -0.06654087819679154),
 (29, 0.03156817445540518),
 (30, 0.041052756028591646),
 (31, 0.045662976308154746),
 (32, -0.07443458108499741),
 (33, 0.046119855821482356),
 (34, -0.016

\#### Please do not change this. This cell is used for grading.

---
**Implementation (5 points):**
 Next, implement a basic ranking class for vector space retrieval (used for all semantic methods): 

In [390]:
# TODO: Implement this! (5 points)
class DenseRetrievalRanker:
    def __init__(self, vsrm, similarity_fn):
        """
            vsrm: instance of `VectorSpaceRetrievalModel`
            similarity_fn: function instance that takes in two vectors 
                            and returns a similarity score e.g cosine_sim defined earlier
        """
        self.vsrm = vsrm 
        self.vectorized_documents = self.vsrm.vectorize_documents()
        self.similarity_fn = similarity_fn
    
    def _compute_sim(self, query_vector):
        """
            Compute the similarity of `query_vector` to documents in 
            `self.vectorized_documents` using `self.similarity_fn`
            Returns a list of (doc_id, score) tuples
        """
        # YOUR CODE HERE
        
        output = []
        
        for doc_id in list(self.vectorized_documents.keys()):
            output.append((doc_id, self.similarity_fn(query_vector, self.vectorized_documents[doc_id])))
          
        # this replaces nans with 0.0 in later checks
        if np.isnan(output[0][1]):
            return [(tup[0], 0.0) for tup in output]
        
        return output
    
    def search(self, query):
        scores = self._compute_sim(self.vsrm.vectorize_query(query))
        scores.sort(key=lambda _:-_[1])
        return scores 


In [80]:
##### Function check
drm_lsi = DenseRetrievalRanker(lsi, cosine_sim)
drm_lsi.search("report")[:5]
##### 

[('599', 0.7942918773386028),
 ('947', 0.5832786688508904),
 ('53', 0.5053737924578612),
 ('1339', 0.44582409698999764),
 ('3160', 0.4450781103545025)]

\#### Please do not change this. This cell is used for grading.

---
Now, you can test your LSI model in the following cell: try finding queries which are lexically different to documents, but semantically similar - does LSI work well for these queries?!

In [81]:
# test your LSI model
search_fn = drm_lsi.search

text = widgets.Text(description="Search Bar", width=200)
display(text)

def make_results_2(query, search_fn):
    results = []
    for doc_id, score in search_fn(query):
        highlight = highlight_text(docs_by_id[doc_id], query)
        if len(highlight.strip()) == 0:
            highlight = docs_by_id[doc_id]
        results.append(ResultRow(doc_id, highlight, score))
    return results

def handle_submit_2(sender):
    print(f"Searching for: '{sender.value}' (SEARCH FN: {search_fn})")
    
    results = make_results_2(sender.value, search_fn)
    
    # display only the top 5
    results = results[:5]
    
    body = ""
    for idx, r in enumerate(results):
        body += f"<li>Document #{r.doc_id}({r.score}): {r.snippet}</li>"
    display(HTML(f"<ul>{body}</ul>"))
    

text.on_submit(handle_submit_2)

Text(value='', description='Search Bar')

---
## Section 7: Latent Dirichlet Allocation (LDA) (10 points) <a class="anchor" id="lda"></a>

[Back to Part 2](#part2)

The specifics of LDA is out of the scope of this assignment, but we will use the `gensim` implementation to perform search using LDA over our small document collection. The key thing to remember is that LDA, unlike LSI, outputs a topic **distribution**, not a vector. With that in mind, let's first define a similarity measure.


---
### Section 7.1: Jenson-Shannon divergence (5 points) <a class="anchor" id="js_sim"></a>

The Jenson-Shannon divergence is a symmetric and finite measure on two probability distributions (unlike the KL, which is neither). For identical distributions, the JSD is equal to 0, and since our code uses 0 as irrelevant and higher scores as relevant, we use `(1 - JSD)` as the score or 'similarity' in our setup

**Note**: the JSD is bounded to \[0,1\] only if we use log base 2. So please ensure that you're using `np.log2` instead of `np.log`

In [82]:
## TODO: Implement this! (5 points)
def jenson_shannon_divergence(vec_1, vec_2, assert_prob=False):
    """
        Computes the Jensen-Shannon divergence between two probability distributions. 
        NOTE: DO NOT RETURN 1 - JSD here, that is handled by the next function which is already implemented! 
        The inputs are *gensim* vectors - same as the vectors for the cosine_sim function
        assert_prob is a flag that checks if the inputs are proper probability distributions 
            i.e they sum to 1 and are positive - use this to check your inputs if needed. 
                (This is optional to implement, but recommended - 
                you can the default to False to save a few ms off the runtime)
    """
    # YOUR CODE HERE
    
    def kl_divergence(vec_1, vec_2):
        
        score = 0
        
        for i in range(len(vec_1)):
            # np.log2 [0, 1]
            # vec_2 is a list and not list of tuples
            score += vec_1[i][1] * np.log2(vec_1[i][1] / vec_2[i])
        
        return score
        
    if assert_prob == True:
        
        # check that all values or positive
        minus_1 = [tup[1] for tup in vec_1 if tup[1] < 0]
        minus_2 = [tup[1] for tup in vec_2 if tup[1] < 0]
        
        if len(minus_1) > 0 or len(minus_2) > 0:
            return 1
        
        # check that the sum is 1
        condition_1 = np.sum([tup[1] for tup in vec_1]) 
        condition_2 = np.sum([tup[1] for tup in vec_2])
    
        if condition_1 == 1 and condition_2 == 1:
            m = [0.5 * (vec_1[i][1] + vec_2[i][1]) for i in range(len(vec_1))] 
            return (0.5 * kl_divergence(vec_1, m)) + (0.5 * kl_divergence(vec_2, m))
        else:
            return 1  
        
    m = [0.5 * (vec_1[i][1] + vec_2[i][1]) for i in range(len(vec_1))] 
    return (0.5 * kl_divergence(vec_1, m)) + (0.5 * kl_divergence(vec_2, m))

def jenson_shannon_sim(vec_1, vec_2, assert_prob=True):
    
    # added the assert_prob to check if the assert_prob in def jenson_shannon_divergence works
    return 1 - jenson_shannon_divergence(vec_1, vec_2, assert_prob)


In [83]:
##### Function check
vec_1 = [(1, 0.3), (2, 0.4), (3, 0.3)]
vec_2 = [(1, 0.1), (2, 0.7), (3, 0.2)]
jenson_shannon_sim(vec_1, vec_2, assert_prob=True)
##### 

0.9251064410358459

---
### Section 7.2: LDA retrieval (5 points) <a class="anchor" id="lda_ret"></a>

Implement the `train_model` method in the following class (note that this is only one line of code in `gensim`!). Ensure that the parameters defined in the `__init__` method are not changed, and are *used in the `train_method` function*. You do not need to set this. Normally, the hyperaparameter space will be searched using grid search / other methods. Note that training the LDA model might take some time

The last two lines of code train an LDA model on the list of documents which have been stemmed, lower-cased and have stopwords removed. 

In [84]:
# TODO: Implement this! (5 points)
class LdaRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        # use these parameters in the train_model method
        self.num_topics = 100
        self.chunksize = 2000
        self.passes = 20
        self.iterations = 400
        self.eval_every = 10
        # this is need to get full vectors
        self.minimum_probability=0.0
        self.alpha='auto'
        self.eta='auto'
    
    
    def train_model(self):
        # YOUR CODE HERE

        self.model = LdaModel(corpus=self.corpus, num_topics=self.num_topics, id2word=self.id2word,
                             chunksize=self.chunksize, passes=self.passes, iterations=self.iterations,
                             eval_every=self.eval_every, minimum_probability=self.minimum_probability,
                             alpha=self.alpha, eta=self.eta)
        
        return self.model

In [85]:
##### Function check
lda = LdaRetrievalModel(doc_repr_2)
lda.train_model()

# you can now get an LDA vector for a given query in the following way:
lda.vectorize_query("report")
##### 

2022-02-28 12:45:22,764 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-02-28 12:45:22,854 : INFO : built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115969 corpus positions)
2022-02-28 12:45:22,858 : INFO : discarding 4740 tokens: [('repeat', 8), ('glossari', 7), ('inspect', 8), ('uncol', 2), ('rung', 9), ('secant', 2), ('.', 1603), ('acceler', 6), ('diverg', 3), ('induc', 9)]...
2022-02-28 12:45:22,859 : INFO : keeping 1197 tokens which were in no less than 10 and no more than 1602 (=50.0%) documents
2022-02-28 12:45:22,860 : INFO : resulting dictionary: Dictionary(1197 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...)
2022-02-28 12:45:22,911 : INFO : using autotuned alpha, starting with [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0

2022-02-28 12:45:25,385 : INFO : merging changes from 2000 documents into a model of 3204 documents
2022-02-28 12:45:25,388 : INFO : topic #90 (0.009): 0.001*"implic" + 0.001*"favor" + 0.001*"adapt" + 0.001*"polygon" + 0.001*"stage" + 0.001*"situat" + 0.001*"respons" + 0.001*"industri" + 0.001*"regress" + 0.001*"drive"
2022-02-28 12:45:25,389 : INFO : topic #25 (0.010): 0.111*"reconstruct" + 0.110*"move" + 0.093*"measur" + 0.072*"intersect" + 0.071*"track" + 0.069*"method" + 0.054*"gener" + 0.054*"," + 0.045*"common" + 0.040*"point"
2022-02-28 12:45:25,389 : INFO : topic #4 (0.011): 0.112*"(" + 0.087*")" + 0.085*"]" + 0.069*"set" + 0.042*"1" + 0.035*"equat" + 0.035*"," + 0.033*"partit" + 0.032*"algorithm" + 0.027*"solut"
2022-02-28 12:45:25,389 : INFO : topic #27 (0.011): 0.233*"(" + 0.231*"algorithm" + 0.217*")" + 0.046*"$" + 0.035*"invers" + 0.031*"))" + 0.027*"-" + 0.026*"ii" + 0.025*"matrix" + 0.021*"normal"
2022-02-28 12:45:25,389 : INFO : topic #33 (0.011): 0.105*"," + 0.030*"-" 

2022-02-28 12:45:27,492 : INFO : topic #90 (0.009): 0.001*"implic" + 0.001*"favor" + 0.001*"adapt" + 0.001*"polygon" + 0.001*"stage" + 0.001*"situat" + 0.001*"respons" + 0.001*"industri" + 0.001*"regress" + 0.001*"drive"
2022-02-28 12:45:27,493 : INFO : topic #25 (0.010): 0.200*"measur" + 0.161*"move" + 0.120*"reconstruct" + 0.071*"intersect" + 0.064*"track" + 0.052*"," + 0.051*"common" + 0.043*"point" + 0.039*"method" + 0.033*"gener"
2022-02-28 12:45:27,493 : INFO : topic #4 (0.012): 0.130*"(" + 0.121*")" + 0.119*"]" + 0.068*"1" + 0.055*"set" + 0.043*"2" + 0.030*"," + 0.029*"[" + 0.027*"partit" + 0.026*"algorithm"
2022-02-28 12:45:27,493 : INFO : topic #33 (0.012): 0.118*"," + 0.033*"comput" + 0.031*"program" + 0.030*"system" + 0.026*"-" + 0.009*"paper" + 0.009*"provid" + 0.009*"develop" + 0.009*"data" + 0.008*"problem"
2022-02-28 12:45:27,494 : INFO : topic #27 (0.013): 0.296*"(" + 0.249*")" + 0.241*"algorithm" + 0.046*"$" + 0.036*"))" + 0.021*"-" + 0.017*"invers" + 0.016*"matrix" + 

2022-02-28 12:45:29,041 : INFO : topic #34 (0.010): 0.081*"architectur" + 0.057*"," + 0.041*"smaller" + 0.040*"system" + 0.031*"consider" + 0.030*"cover" + 0.028*"entir" + 0.027*"paper" + 0.024*"past" + 0.021*"data"
2022-02-28 12:45:29,042 : INFO : topic #4 (0.013): 0.144*"(" + 0.132*")" + 0.092*"]" + 0.082*"1" + 0.061*"2" + 0.049*"set" + 0.029*"," + 0.029*"3" + 0.028*"partit" + 0.026*"["
2022-02-28 12:45:29,042 : INFO : topic #33 (0.013): 0.128*"," + 0.036*"comput" + 0.033*"program" + 0.029*"system" + 0.024*"-" + 0.010*"provid" + 0.010*"develop" + 0.010*"paper" + 0.009*"problem" + 0.009*"data"
2022-02-28 12:45:29,042 : INFO : topic #27 (0.016): 0.320*"(" + 0.284*")" + 0.238*"algorithm" + 0.028*"$" + 0.020*"))" + 0.019*"integr" + 0.014*"-" + 0.014*"invers" + 0.014*"matrix" + 0.010*"normal"
2022-02-28 12:45:29,043 : INFO : topic diff=0.594575, rho=0.389191
2022-02-28 12:45:29,484 : INFO : -6.634 per-word bound, 99.3 perplexity estimate based on a held-out corpus of 1204 documents with 4

2022-02-28 12:45:30,897 : INFO : topic #20 (0.013): 0.232*"[" + 0.207*"algorithm" + 0.163*"(" + 0.121*"])" + 0.023*"complex" + 0.019*"g6" + 0.018*"matrix" + 0.018*"function" + 0.016*"eigenvalu" + 0.016*"gener"
2022-02-28 12:45:30,897 : INFO : topic #4 (0.014): 0.161*"(" + 0.159*")" + 0.108*"]" + 0.090*"1" + 0.062*"2" + 0.037*"set" + 0.033*"[" + 0.030*"3" + 0.026*"partit" + 0.026*","
2022-02-28 12:45:30,898 : INFO : topic #27 (0.020): 0.339*"(" + 0.288*")" + 0.230*"algorithm" + 0.033*"$" + 0.026*"))" + 0.016*"integr" + 0.013*"-" + 0.012*"matrix" + 0.009*"invers" + 0.008*"normal"
2022-02-28 12:45:30,898 : INFO : topic diff=0.625042, rho=0.362690
2022-02-28 12:45:30,902 : INFO : PROGRESS: pass 6, at document #2000/3204
2022-02-28 12:45:31,250 : INFO : optimized alpha [0.011893852, 0.010285689, 0.010619514, 0.010738807, 0.013837697, 0.010713068, 0.011246421, 0.009844524, 0.010755475, 0.010264293, 0.0105744675, 0.010520432, 0.011175848, 0.010845604, 0.011013993, 0.011036047, 0.0104912855, 0

...

2022-02-28 12:45:40,541 : INFO : topic #34 (0.010): 0.080*"consider" + 0.073*"architectur" + 0.064*"," + 0.058*"cover" + 0.045*"paper" + 0.039*"system" + 0.037*"entir" + 0.035*"smaller" + 0.022*"past" + 0.021*"deriv"
2022-02-28 12:45:40,542 : INFO : topic #35 (0.019): 0.240*"system" + 0.089*"-" + 0.067*"," + 0.060*"user" + 0.046*"comput" + 0.042*"time" + 0.041*"oper" + 0.040*"share" + 0.028*"process" + 0.026*"design"
2022-02-28 12:45:40,542 : INFO : topic #20 (0.020): 0.235*"[" + 0.226*"algorithm" + 0.173*"(" + 0.148*"])" + 0.023*"complex" + 0.019*"matrix" + 0.017*"g6" + 0.017*"-" + 0.014*"eigenvalu" + 0.013*"symmetr"
2022-02-28 12:45:40,542 : INFO : topic #27 (0.050): 0.368*"(" + 0.318*")" + 0.218*"algorithm" + 0.025*"$" + 0.019*"))" + 0.010*"-" + 0.010*"integr" + 0.007*"normal" + 0.006*"gauss" + 0.006*"s15"
2022-02-28 12:45:40,543 : INFO : topic diff=0.189529, rho=0.238352
2022-02-28 12:45:40,938 : INFO : -6.453 per-word bound, 87.6 perplexity estimate based on a held-out corpus of 1

2022-02-28 12:45:42,211 : INFO : topic #35 (0.020): 0.243*"system" + 0.090*"-" + 0.070*"," + 0.061*"user" + 0.045*"comput" + 0.041*"oper" + 0.040*"time" + 0.039*"share" + 0.027*"design" + 0.027*"process"
2022-02-28 12:45:42,212 : INFO : topic #20 (0.021): 0.244*"[" + 0.235*"algorithm" + 0.166*"(" + 0.125*"])" + 0.026*"complex" + 0.021*"matrix" + 0.017*"g6" + 0.017*"-" + 0.015*"eigenvalu" + 0.014*"symmetr"
2022-02-28 12:45:42,212 : INFO : topic #27 (0.055): 0.375*"(" + 0.315*")" + 0.213*"algorithm" + 0.029*"$" + 0.023*"))" + 0.010*"-" + 0.007*"normal" + 0.006*"integr" + 0.006*"gauss" + 0.005*"s15"
2022-02-28 12:45:42,212 : INFO : topic diff=0.178929, rho=0.231857
2022-02-28 12:45:42,216 : INFO : PROGRESS: pass 17, at document #2000/3204
2022-02-28 12:45:42,532 : INFO : optimized alpha [0.015200603, 0.011407376, 0.012735999, 0.012673991, 0.019930992, 0.012896201, 0.017362112, 0.010943501, 0.013357442, 0.011629898, 0.01255746, 0.011880305, 0.012706592, 0.012764518, 0.01311403, 0.013923637

2022-02-28 12:45:43,533 : INFO : topic #20 (0.022): 0.237*"[" + 0.226*"algorithm" + 0.173*"(" + 0.146*"])" + 0.023*"complex" + 0.019*"matrix" + 0.017*"g6" + 0.017*"-" + 0.014*"eigenvalu" + 0.014*"symmetr"
2022-02-28 12:45:43,534 : INFO : topic #27 (0.061): 0.375*"(" + 0.321*")" + 0.217*"algorithm" + 0.025*"$" + 0.019*"))" + 0.010*"-" + 0.008*"normal" + 0.006*"gauss" + 0.006*"s15" + 0.004*"integr"
2022-02-28 12:45:43,534 : INFO : topic diff=0.151173, rho=0.220316
2022-02-28 12:45:43,927 : INFO : -6.436 per-word bound, 86.6 perplexity estimate based on a held-out corpus of 1204 documents with 49783 words
2022-02-28 12:45:43,927 : INFO : PROGRESS: pass 18, at document #3204/3204
2022-02-28 12:45:44,192 : INFO : optimized alpha [0.015683018, 0.011522268, 0.01317574, 0.012937769, 0.020791918, 0.013365951, 0.018353058, 0.011179138, 0.013719281, 0.011865424, 0.012876664, 0.012151176, 0.012955175, 0.013065434, 0.013489797, 0.014396328, 0.011769344, 0.01162892, 0.012950082, 0.011222547, 0.02265

2022-02-28 12:45:45,197 : INFO : topic #27 (0.066): 0.379*"(" + 0.318*")" + 0.213*"algorithm" + 0.029*"$" + 0.022*"))" + 0.010*"-" + 0.007*"normal" + 0.006*"gauss" + 0.005*"s15" + 0.003*"ii"
2022-02-28 12:45:45,197 : INFO : topic diff=0.145060, rho=0.215156


[(0, 0.0065119434),
 (1, 0.0047465367),
 (2, 0.0054778755),
 (3, 0.005355076),
 (4, 0.008676807),
 (5, 0.0055518085),
 (6, 0.0077367993),
 (7, 0.0046235975),
 (8, 0.0056984364),
 (9, 0.004897008),
 (10, 0.0053288587),
 (11, 0.0050218725),
 (12, 0.0053431736),
 (13, 0.0054019545),
 (14, 0.0055867126),
 (15, 0.005986876),
 (16, 0.004847735),
 (17, 0.004797258),
 (18, 0.005344449),
 (19, 0.0046113143),
 (20, 0.0095156245),
 (21, 0.00581009),
 (22, 0.004563651),
 (23, 0.005468314),
 (24, 0.41286823),
 (25, 0.0043050125),
 (26, 0.0055397362),
 (27, 0.026751583),
 (28, 0.0063317264),
 (29, 0.006338123),
 (30, 0.005706597),
 (31, 0.005139023),
 (32, 0.0056577455),
 (33, 0.007176503),
 (34, 0.004323071),
 (35, 0.009289766),
 (36, 0.0051612207),
 (37, 0.006056151),
 (38, 0.005590305),
 (39, 0.0056708413),
 (40, 0.0058165803),
 (41, 0.007881726),
 (42, 0.006064678),
 (43, 0.0054300134),
 (44, 0.0051501133),
 (45, 0.0055146907),
 (46, 0.0057616914),
 (47, 0.005687281),
 (48, 0.006336866),
 (49, 0

\#### Please do not change this. This cell is used for grading.

---
Now we can use the `DenseRetrievalModel` class to obtain an LDA search function.
You can test your LDA model in the following cell: Try finding queries which are lexically different to documents, but semantically similar - does LDA work well for these queries?!

In [86]:
drm_lda = DenseRetrievalRanker(lda, jenson_shannon_sim)

# test your LDA model
search_fn = drm_lda.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

Text(value='', description='Search Bar')

## Section 8: Word2Vec/Doc2Vec (20 points) <a class="anchor" id="2vec"></a>

[Back to Part 2](#part2)

We will implement two other methods here, the Word2Vec model and the Doc2Vec model, also using `gensim`. Word2Vec creates representations of words, not documents, so the word level vectors need to be aggregated to obtain a representation for the document. Here, we will simply take the mean of the vectors. 


A drawback of these models is that they need a lot of training data. Our dataset is tiny, so in addition to using a model trained on the data, we will also use a pre-trained model for Word2Vec (this will be automatically downloaded).     

*Note*:
1. The code in vectorize_documents / vectorize_query should return gensim-like vectors i.e `[(dim, val), .. (dim, val)]`. 
2. For Word2Vec: You should also handle the following two cases: (a) A word in the query is not present in the vocabulary of the model and (b) none of the words in the query are present in the model - you can return 0 scores for all documents in this case. For either of these, you can check if a `word` is present in the vocab by using `word in self.model`


In [391]:
# TODO: Implement this! (10 points)
class W2VRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        # the dimensionality of the vectors
        self.size = 100 
        self.min_count = 1
    
    def train_model(self):
        """
        Trains the W2V model
        """
        # YOUR CODE HERE

        self.model = Word2Vec(sentences=self.documents, size=self.size, min_count=self.min_count)
    
        return self.model
        
    def vectorize_documents(self):
        """
            Returns a doc_id -> vector dictionary
        """
        
        # YOUR CODE HERE        
        
        vectors = dict()
        
        for (doc_id, _), cc in zip(self.doc_repr, self.documents):

            # this check is needed for the last check before doc2vec
            cc = [word for word in cc if word in self.model]
            
            if len(cc) != 0:
                output = np.mean(self.model[cc], axis=0)
            else:
                output = np.zeros(self.size)
                
            # same format as query vectors
            vectors[doc_id] = [(x, output[x]) for x in range(self.size)]

        return vectors
    

    def vectorize_query(self, query):
        """
        Vectorizes the query using the W2V model
        """
        query = process_text(query, **config_2)
        # YOUR CODE HERE

        filtered_query = []
        
        # skip the words that are not in self.model --> ranking will still be the same
        for word in query:
            if word in self.model:
                filtered_query.append(word)
                
        if len(filtered_query) == 0:
            return [(x, 0.0) for x in range(self.size)]
            
        vectorized_query = np.mean(self.model[filtered_query], axis=0)
        
        return [(x, vectorized_query[x]) for x in range(self.size)]

class W2VPretrainedRetrievalModel(W2VRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        self.model_name = "word2vec-google-news-300"
        self.size = 300
    
    def train_model(self):
        """
        Loads the pretrained model
        """
        self.model = g_downloader.load(self.model_name)

w2v = W2VRetrievalModel(doc_repr_2)
w2v.train_model()

# you can now get a W2V vector for a given query in the following way:
w2v.vectorize_query("report")

2022-02-28 15:16:22,541 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-02-28 15:16:22,640 : INFO : built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115969 corpus positions)
2022-02-28 15:16:22,644 : INFO : discarding 4740 tokens: [('repeat', 8), ('glossari', 7), ('inspect', 8), ('uncol', 2), ('rung', 9), ('secant', 2), ('.', 1603), ('acceler', 6), ('diverg', 3), ('induc', 9)]...
2022-02-28 15:16:22,644 : INFO : keeping 1197 tokens which were in no less than 10 and no more than 1602 (=50.0%) documents
2022-02-28 15:16:22,646 : INFO : resulting dictionary: Dictionary(1197 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...)
2022-02-28 15:16:22,699 : INFO : collecting all words and their counts
2022-02-28 15:16:22,699 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-02-28 15:16:22,712 : INFO : collected 5937 word types from a corpus of 115969 raw w

[(0, 0.48815623),
 (1, 0.06465161),
 (2, 0.11326133),
 (3, 0.42731804),
 (4, -0.6913425),
 (5, 0.17683126),
 (6, -0.04831889),
 (7, -0.07894417),
 (8, -0.055554535),
 (9, 0.6640186),
 (10, -0.2591971),
 (11, -0.5924829),
 (12, -0.31710362),
 (13, -0.7106117),
 (14, 0.50445276),
 (15, 0.2760345),
 (16, 0.580353),
 (17, 0.7635267),
 (18, -0.64346117),
 (19, -0.024600744),
 (20, -0.19223449),
 (21, -0.34653082),
 (22, -0.65606457),
 (23, -0.5759722),
 (24, -0.13997929),
 (25, 0.4436178),
 (26, -0.57044166),
 (27, -0.59964234),
 (28, 0.0760233),
 (29, -0.109359466),
 (30, -0.16681032),
 (31, -0.071641274),
 (32, 0.28639722),
 (33, 0.23439457),
 (34, 0.57399666),
 (35, -0.4120237),
 (36, -0.031251274),
 (37, -0.32252064),
 (38, -0.31436822),
 (39, -0.077925645),
 (40, 0.21055561),
 (41, -0.16324574),
 (42, 1.0324949),
 (43, 0.16151407),
 (44, -0.08067311),
 (45, 0.27689192),
 (46, 0.064418934),
 (47, 0.008828646),
 (48, -0.09241296),
 (49, -0.21006452),
 (50, -0.037817735),
 (51, -0.5250468

In [392]:
assert len(w2v.vectorize_query("report")) == 100
assert len(w2v.vectorize_query("this is a sentence that is not mellifluous")) == 100


/Users/kevintran/opt/anaconda3/envs/python365/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/kevintran/opt/anaconda3/envs/python365/lib/python3.6/site-packages/ipykernel_launcher.py:62: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


\#### Please do not change this. This cell is used for grading.

In [393]:
w2v_pretrained = W2VPretrainedRetrievalModel(doc_repr_2)
w2v_pretrained.train_model()

# you can now get an W2V vector for a given query in the following way:
w2v_pretrained.vectorize_query("report")

2022-02-28 15:16:32,090 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-02-28 15:16:32,208 : INFO : built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115969 corpus positions)
2022-02-28 15:16:32,214 : INFO : discarding 4740 tokens: [('repeat', 8), ('glossari', 7), ('inspect', 8), ('uncol', 2), ('rung', 9), ('secant', 2), ('.', 1603), ('acceler', 6), ('diverg', 3), ('induc', 9)]...
2022-02-28 15:16:32,215 : INFO : keeping 1197 tokens which were in no less than 10 and no more than 1602 (=50.0%) documents
2022-02-28 15:16:32,216 : INFO : resulting dictionary: Dictionary(1197 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...)
2022-02-28 15:16:32,336 : INFO : loading projection weights from /Users/kevintran/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz
2022-02-28 15:17:21,427 : INFO : loaded (3000000, 300) matrix from /Users/kevintran/gensim-data/word2vec-google

[(0, -0.14257812),
 (1, -0.1640625),
 (2, -0.09033203),
 (3, -0.11230469),
 (4, 0.100097656),
 (5, -0.041259766),
 (6, 0.048828125),
 (7, -0.13671875),
 (8, 0.19628906),
 (9, -0.13476562),
 (10, -0.017578125),
 (11, 0.032226562),
 (12, 0.095214844),
 (13, -0.10595703),
 (14, -0.16992188),
 (15, 0.041015625),
 (16, -0.26367188),
 (17, -0.0063171387),
 (18, -0.17773438),
 (19, -0.24023438),
 (20, 0.3515625),
 (21, -0.012207031),
 (22, -0.16210938),
 (23, -0.12060547),
 (24, 0.04321289),
 (25, 0.10986328),
 (26, 0.052490234),
 (27, 0.17871094),
 (28, -0.14550781),
 (29, 0.13769531),
 (30, -0.08203125),
 (31, -0.28320312),
 (32, -0.10888672),
 (33, -0.2890625),
 (34, 0.072265625),
 (35, -0.04736328),
 (36, 0.040283203),
 (37, 0.067871094),
 (38, 0.11669922),
 (39, 0.000831604),
 (40, 0.068359375),
 (41, 0.12011719),
 (42, -0.088378906),
 (43, 0.33789062),
 (44, -0.044677734),
 (45, -0.030151367),
 (46, 0.0076904297),
 (47, -0.021118164),
 (48, -0.25390625),
 (49, 0.14941406),
 (50, 0.39843

In [394]:
##### Function check

print(len(w2v_pretrained.vectorize_query("report")))
#####

300


In [395]:
drm_w2v = DenseRetrievalRanker(w2v, cosine_sim)

# test your LDA model
search_fn = drm_w2v.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

/Users/kevintran/opt/anaconda3/envs/python365/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/kevintran/opt/anaconda3/envs/python365/lib/python3.6/site-packages/ipykernel_launcher.py:35: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Text(value='', description='Search Bar')

Searching for: 'kaas' (SEARCH FN: <bound method DenseRetrievalRanker.search of <__main__.DenseRetrievalRanker object at 0x7fc0c143d6a0>>)


/Users/kevintran/opt/anaconda3/envs/python365/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/kevintran/opt/anaconda3/envs/python365/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars


Searching for: '' (SEARCH FN: <bound method DenseRetrievalRanker.search of <__main__.DenseRetrievalRanker object at 0x7fc0c143d6a0>>)


/Users/kevintran/opt/anaconda3/envs/python365/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars


In [281]:
drm_w2v_pretrained = DenseRetrievalRanker(w2v_pretrained, cosine_sim)

# test your LDA model
search_fn = drm_w2v_pretrained.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

Text(value='', description='Search Bar')

Searching for: 'computer' (SEARCH FN: <bound method DenseRetrievalRanker.search of <__main__.DenseRetrievalRanker object at 0x7fc234283668>>)


/Users/kevintran/opt/anaconda3/envs/python365/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars


Searching for: 'kaas' (SEARCH FN: <bound method DenseRetrievalRanker.search of <__main__.DenseRetrievalRanker object at 0x7fc234283668>>)


/Users/kevintran/opt/anaconda3/envs/python365/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars


**Implementation (10 points):**
For Doc2Vec, you will need to create a list of `TaggedDocument` instead of using the `self.corpus` or `self.documents` variable. Use the document id as the 'tag'.
  

In [379]:
# delete this line after the code works
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# TODO: Implement this! (10 points)
class D2VRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        self.vector_size= 100
        self.min_count = 1
        self.epochs = 20
        
        # YOUR CODE HERE
        
        self.TaggedDocument = []

        for (doc_id, _), cc in zip(self.doc_repr, self.documents):
            self.TaggedDocument.append(TaggedDocument(cc, [doc_id]))
        
    def train_model(self):
        # YOUR CODE HERE
        
        self.model = Doc2Vec(documents=self.TaggedDocument, vector_size=self.vector_size, min_count=self.min_count,
                            epochs=self.epochs)

        return self.model
    
    def vectorize_documents(self):
        """
            Returns a doc_id -> vector dictionary
        """
        # YOUR CODE HERE
       
        vectors = dict()
        
        for words, tag in self.TaggedDocument:
            
            output = np.mean([self.model[x] for x in words], axis=0)
            print(output)
            
            vectors[tag[0]] = [(x, output[x]) for x in range(self.vector_size)]

        return vectors

    def vectorize_query(self, query):
        
        # YOUR CODE HERE
        
        query = process_text(query, **config_2)
        
        vectorized_query = np.mean(self.model[query], axis=0)

        output = [(i, vectorized_query[i]) for i in range(self.vector_size)]
        
        return output
    
        
d2v = D2VRetrievalModel(doc_repr_2)
d2v.train_model()


# # you can now get an LSI vector for a given query in the following way:
d2v.vectorize_query("computer")

2022-02-28 14:59:47,091 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-02-28 14:59:47,200 : INFO : built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115969 corpus positions)
2022-02-28 14:59:47,205 : INFO : discarding 4740 tokens: [('repeat', 8), ('glossari', 7), ('inspect', 8), ('uncol', 2), ('rung', 9), ('secant', 2), ('.', 1603), ('acceler', 6), ('diverg', 3), ('induc', 9)]...
2022-02-28 14:59:47,205 : INFO : keeping 1197 tokens which were in no less than 10 and no more than 1602 (=50.0%) documents
2022-02-28 14:59:47,207 : INFO : resulting dictionary: Dictionary(1197 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...)
2022-02-28 14:59:47,264 : INFO : collecting all words and their counts
2022-02-28 14:59:47,265 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2022-02-28 14:59:47,282 : INFO : collected 5937 word types and 3204 unique tags fro

2022-02-28 14:59:51,394 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-02-28 14:59:51,394 : INFO : EPOCH - 16 : training on 115969 raw words (95634 effective words) took 0.2s, 544047 effective words/s
2022-02-28 14:59:51,563 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-02-28 14:59:51,569 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-02-28 14:59:51,574 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-02-28 14:59:51,574 : INFO : EPOCH - 17 : training on 115969 raw words (95623 effective words) took 0.2s, 538603 effective words/s
2022-02-28 14:59:51,751 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-02-28 14:59:51,757 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-02-28 14:59:51,760 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-02-28 14:59:51,760 : INFO : EPOCH - 18 : training on 115969 raw words (95638 effective w

[(0, 0.50772625),
 (1, 0.490202),
 (2, 0.33463788),
 (3, 0.7252605),
 (4, -0.20228855),
 (5, -0.29875687),
 (6, -0.06708133),
 (7, -0.8339634),
 (8, 0.35502377),
 (9, 0.17484136),
 (10, 0.86556745),
 (11, 0.0074797114),
 (12, -0.32112512),
 (13, -1.0198058),
 (14, 0.6270973),
 (15, -0.51996624),
 (16, 0.91890913),
 (17, 0.9532468),
 (18, 0.29471177),
 (19, -0.2340536),
 (20, -0.7116523),
 (21, 0.6191798),
 (22, 0.22497962),
 (23, -0.34936294),
 (24, -0.34658098),
 (25, 0.010128347),
 (26, -0.67233306),
 (27, -0.13048285),
 (28, 0.38019842),
 (29, 0.023216618),
 (30, 0.91715986),
 (31, 0.30551755),
 (32, -0.31052896),
 (33, 0.5144318),
 (34, -0.2891626),
 (35, -0.58698344),
 (36, 0.052316118),
 (37, -0.8336487),
 (38, -0.6968471),
 (39, 0.15445504),
 (40, 0.13710664),
 (41, -0.1101188),
 (42, 0.544456),
 (43, -0.38576442),
 (44, -0.20916052),
 (45, 0.7235959),
 (46, -0.00030902392),
 (47, 0.054365437),
 (48, 0.345175),
 (49, -0.039070938),
 (50, 0.12039261),
 (51, 0.023985654),
 (52, 0.

In [376]:
#### Please do not change this. This cell is used for grading.

\#### Please do not change this. This cell is used for grading.

In [380]:
drm_d2v = DenseRetrievalRanker(d2v, cosine_sim)

# test your LDA model
search_fn = drm_d2v.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

[ 0.45878038  0.57950306  0.14796065  0.28329542 -0.27628955  0.6212384
  0.11004105 -0.69193697  0.4705771   0.40616882 -0.16360448 -0.5195549
 -0.20259972 -0.7842848   0.25909904  0.17088278  0.7050236   0.6002701
 -0.11880659 -0.41934228  0.13742976  0.26207647 -0.3088306  -0.29280537
  0.06675779  0.8389247  -0.12819965 -0.51750225  0.19411726 -0.2505035
  0.3243742   0.21947789 -0.10372462  0.01269976 -0.00502234 -0.20432973
  0.2932119  -0.4537853  -0.8742595   0.19692129 -0.13221362 -0.05147515
  0.78023964 -0.19390495 -0.15064697  0.16386327  0.04326168 -0.23886622
  0.31506664 -0.10382959  0.24862342 -0.51874346  0.00692108 -0.2357047
  0.07120102  0.4444499  -0.16266714 -0.48396787 -0.4603751   0.03495976
 -0.31216523  0.2609514  -0.3747284   0.6986344  -0.44279107 -0.36320874
 -0.25030422  0.9594774  -0.01497408  0.9719429  -0.0736578  -0.48890767
  0.20123518 -0.7994644   0.48980045  0.11205187 -0.77850276  0.00794892
 -0.05001801 -0.08834755 -0.01550925  0.19034438 -0.2283

 -2.8353032e-01  2.9429802e-01 -3.1244603e-01 -1.9431295e-01]
[ 0.49128994  0.38536388  0.29051244  0.28668135 -0.4229836   0.580268
  0.03290438  0.0819978   0.0524745   0.42375553 -0.26609716 -0.14678648
 -0.43134364 -0.52057385  0.2769833  -0.13757676  0.4199086   0.5968252
 -0.18195808 -0.09175155 -0.1246619   0.4185027  -0.1977182  -0.312012
  0.1391898   0.30267    -0.11655746 -0.28695306  0.6144311  -0.00326333
 -0.05147479  0.10428799 -0.20896317  0.01196088  0.35046542 -0.09467831
 -0.01293896 -0.07972011 -0.03241228  0.35964495  0.15909587  0.03006808
  0.64613944  0.08451329  0.2011648   0.3726414  -0.23071742  0.1040841
  0.13174486 -0.05384426 -0.19385092 -0.23883395 -0.04513517  0.02082319
 -0.02531953  0.60828996  0.2199661  -0.21429047 -0.39237243  0.13596877
 -0.15685554  0.04936474 -0.51609975 -0.00791805 -0.29713908 -0.72416437
 -0.24619958  0.40919787 -0.29327336  0.73259085 -0.24047464 -0.07248496
  0.1281088  -0.30856976  0.26134855 -0.03551856 -0.29759005  0.1800

[-0.5836059  -0.6867545  -0.42923325 -0.06915011 -0.42440957 -0.15870732
 -0.3418679   0.13943128 -0.05778778  0.7218083  -0.57338154 -0.66339177
 -0.11751497  0.06208268 -0.30644938  0.6062433   0.04726529  0.18721619
 -0.82811916 -0.02218994  0.101267   -0.4953677  -0.69458866  0.02147149
  0.30969602 -0.03553791 -0.01596284 -0.14421143 -0.14281961  0.15848136
 -0.7072223  -0.05755012  0.44180942  0.08681449 -0.57580817 -0.67204773
 -0.29570663 -0.5195459  -0.4440635   0.17618415  0.52964556  0.05166658
  0.92027473  0.2422069   0.00234215 -0.38145033  0.01733506  0.01134567
 -0.1427097  -0.2886154   0.2931413  -0.32702866  0.6693656  -0.34703964
 -0.3057428   0.02435259  0.45088536  0.07681958 -0.10722003  0.5436324
  0.41165623 -0.0749616  -1.4194877  -0.34925228 -0.18566117 -0.39436072
  0.06391934  0.20136556  0.09149491 -0.21634534  0.05609722 -0.45828047
 -0.13058142  0.02296634  0.08552963 -0.13544275  0.4183465  -0.39893907
 -0.00603254  0.6084572  -0.45251435  0.02906103 -0.

  0.43895832  0.47772413 -0.2869597  -0.2152342 ]
[-0.5349622  -0.7026961  -0.16049165  0.11032046 -0.5933979  -0.51008034
 -0.6002349   0.15450552  0.09996048  0.97763455 -0.58208305 -0.89164954
 -0.531661   -0.05905394 -0.04749356  0.5228932   0.07272445  0.33613965
 -0.9613055  -0.277336   -0.18906933 -0.7592503  -0.48050526  0.315429
  0.20921455 -0.5474216  -0.60348725  0.01623205  0.25641286  0.22716683
 -0.68349296 -0.54827344  0.3966123   0.23732063 -0.49818838 -0.6368854
 -0.45517588 -0.33194193 -0.14502072 -0.16622895  0.47272363  0.20360678
  1.287138    0.70898676  0.11623707 -0.5360045   0.24827583  0.29387012
 -0.21047466 -0.13519244 -0.23412456 -0.42536694  1.1041495   0.17703934
 -0.03450487  0.6201923   0.5749691   0.00572511  0.06266817  0.71710664
  0.88294286 -0.09895276 -1.4939485  -0.35796234 -0.09885311 -0.4894737
  0.0723762   0.6792483   0.02229738 -0.01440541 -0.47473732 -0.14521702
 -0.5453629   0.14496545 -0.4391318  -0.33272123  0.34626627 -0.19088514
  0.0

 -5.5363268e-01 -2.5532198e-01 -4.6685076e-01  2.6859590e-01]
[ 0.22591527  0.0739153  -0.13755862  0.21505575 -0.414244    1.1782513
  0.26263756 -0.8446873  -0.12105708  0.5896499  -0.6455963  -0.71904963
  0.24652031 -0.69959354  0.06745666  0.36494732  0.38131267  0.29453498
 -0.14351474 -0.680491    0.45266208  0.2744544  -0.77695686 -0.361183
  0.38452616  0.9706256  -0.2551753  -0.80637854  0.35226735 -0.6229075
 -0.34830856  0.49998766  0.06608961 -0.12238698  0.17419584 -0.5374724
  0.46247092 -0.50470334 -1.0877573   0.50643873 -0.04784119 -0.10043576
  0.6787497   0.08053856 -0.19027841 -0.14727618  0.5066298  -0.67991316
  0.09918842  0.01901037  0.46018514 -0.7317363   0.2662376  -0.56021965
  0.37054402  0.24020031 -0.29677504 -0.49642706 -0.03031898  0.11277467
 -0.34141618  0.06090002 -0.6911585   0.6590034  -0.33947852 -0.18006597
 -0.15358339  1.1549422   0.39472413  1.0864025   0.10104159 -0.24282332
  0.0293543  -0.82071227  0.35800034  0.05433477 -0.48893207 -0.432

...

[ 0.5059278   0.6388022   0.19276646  0.4027647  -0.3873718   0.62292147
  0.03223096 -0.18144539  0.1880512   0.5647828  -0.07320588 -0.31263658
 -0.21541944 -0.422622    0.3198156  -0.11148612  0.43574804  0.6894422
 -0.08794659 -0.45768818  0.0839734   0.30279547  0.02050346 -0.17166854
  0.17659473  0.27533725 -0.5001658  -0.3164751   0.5479876   0.11722285
  0.11590701  0.19365692 -0.4209739   0.15371265  0.18514222 -0.3738301
  0.03846678 -0.4101137  -0.39340684  0.2696107   0.06710722  0.06059806
  0.6362111  -0.02709883  0.30580026  0.2664112   0.01239108  0.03785297
  0.2461127  -0.15860365 -0.07480872 -0.21699606  0.23626623  0.06775808
 -0.08834922  0.70772827  0.20924526 -0.4461343  -0.15634237  0.39101318
 -0.02212457 -0.06240058 -0.33903423  0.23936886 -0.35672733 -0.4973249
 -0.23527527  0.49126965 -0.09895211  0.90917575 -0.11022533  0.02448807
 -0.00114824 -0.34534046  0.07007931  0.02191068 -0.36036146  0.29746136
  0.03135823  0.16679516 -0.08055348 -0.02103846  0.00

  0.48862764  0.22413154 -0.05639743 -0.04937619]
[ 0.41672876  0.35086134  0.3503278   0.299829   -0.24732703  0.46848848
  0.03488649 -0.10129484  0.33136073  0.25388983 -0.22230645 -0.2572015
 -0.19758783 -0.58503586  0.23881736  0.01921159  0.6595862   0.56167454
 -0.42531502  0.01997382  0.03975204  0.4180091  -0.3616915  -0.22472627
  0.31348538  0.5310002  -0.01589475 -0.26288602  0.5442518  -0.05066679
  0.12202192  0.29921928  0.0768159  -0.27963543  0.1176845   0.02909739
  0.19991899  0.00167916 -0.241608    0.25207946 -0.16364376  0.10527833
  0.6734568  -0.11077812  0.07982916  0.3066562  -0.2587695  -0.13758728
  0.09380575 -0.00623181 -0.06743302 -0.47699627 -0.13712628 -0.24694534
  0.2128597   0.5566468  -0.10793533 -0.26695073 -0.1203126   0.09518532
 -0.20768797  0.20264961 -0.5866576   0.0589624  -0.35103586 -0.43471518
 -0.26237565  0.35679492 -0.43679184  0.7963393  -0.14378335 -0.29088277
  0.13248561 -0.4013549   0.44792813  0.20791335 -0.18857412  0.06373331
 -

[ 0.21618338  0.1561823   0.03306506  0.4204973  -0.22169863  0.19586627
  0.20315507  0.00090339  0.01746349  0.2773812   0.11150604 -0.16878656
 -0.111491   -0.59859097  0.41848764 -0.16175881  0.45340154  0.27654383
 -0.0521602  -0.06716215 -0.29941556  0.16164555 -0.10900073 -0.16068462
  0.13232598  0.18375558 -0.3259     -0.27866843  0.3532681   0.03190555
  0.0855517   0.09739163 -0.28181958  0.04468855 -0.00146508 -0.33973062
  0.14631149 -0.4362438  -0.34210294  0.20761208 -0.01692683 -0.12518474
  0.43492946 -0.1767004  -0.14562996  0.4029668   0.06188871  0.00344185
  0.14091478  0.03880279  0.09064873 -0.22670002  0.1105151   0.12414689
 -0.11262924  0.47612202  0.11675922 -0.21952896 -0.15900424  0.19025828
 -0.12614162  0.24124965 -0.17796639  0.48768753 -0.25823605 -0.22269121
  0.06355798  0.34826037 -0.30628446  0.5413845   0.07684561 -0.1062903
  0.13720101 -0.3238744   0.14266826 -0.12705766 -0.31592277  0.18118863
 -0.09485615 -0.12479087 -0.306272   -0.10656593 -0.

[ 0.1712888   0.14320646  0.04099271  0.31101006 -0.32347688  0.3365034
  0.09020472  0.04433485 -0.00630891  0.5052612  -0.04610938 -0.29529712
 -0.20991896 -0.44243476  0.27481508  0.08917933  0.46987602  0.39786324
 -0.24333224 -0.06811288 -0.05405536  0.15992543 -0.29725924 -0.12959215
  0.22646421  0.22543165 -0.05295411 -0.2841077   0.3121504   0.17929767
 -0.01093951  0.22274384 -0.18366599  0.00838888 -0.00489911 -0.41720092
  0.05614345 -0.47435188 -0.43890783  0.2656386   0.06720956 -0.06104963
  0.5962616  -0.18813126 -0.00770645  0.35278428 -0.03861065 -0.05023894
  0.12391625 -0.0933321   0.1922388  -0.31737193  0.03479673 -0.13894504
 -0.21901327  0.41699207  0.16317521 -0.15155856 -0.19406068  0.18040816
 -0.14659944  0.14809789 -0.6028456   0.34239924 -0.44434127 -0.39772853
 -0.0512615   0.3022067  -0.35472354  0.58632475  0.08752647 -0.28286672
  0.2309964  -0.38439158  0.34201583 -0.05522605 -0.11225547  0.11069579
 -0.07882026  0.03162689 -0.34392023 -0.05404518 -0.

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



---
## Section 9: Re-ranking (10 points) <a class="anchor" id="reranking"></a>

[Back to Part 2](#part2)

To motivate the re-ranking perspective (i.e retrieve with lexical method + rerank with a semantic method), let's search using semantic methods and compare it to BM25's performance, along with their runtime:


In [ ]:
query = "algebraic functions"
print("BM25: ")
%timeit bm25_search(query, 2)
print("LSI: ")
%timeit drm_lsi.search(query)
print("LDA: ")
%timeit drm_lda.search(query)
print("W2V: ")
%timeit drm_w2v.search(query)
print("W2V(Pretrained): ")
%timeit drm_w2v_pretrained.search(query)
print("D2V:")
%timeit drm_d2v.search(query)

---

**Implementation (10 points):**
Re-ranking involves retrieving a small set of documents using simple but fast methods like BM25 and then re-ranking them with the aid of semantic methods such as LDA or LSI. Implement the following class, which takes in an `initial_retrieval_fn` - the initial retrieval function and `vsrm` - an instance of the `VectorSpaceRetrievalModel` class (i.e LSI/LDA) as input. The search function should first retrieve an initial list of K documents, and then these documents are re-ranked using a semantic method. This not only makes retrieval faster, but semantic methods perform poorly when used in isolation, as you will find out.

In [ ]:
# TODO: Implement this! (10 points)
class DenseRerankingModel:
    def __init__(self, initial_retrieval_fn, vsrm, similarity_fn):
        """
            initial_retrieval_fn: takes in a query and returns a list of [(doc_id, score)] (sorted)
            vsrm: instance of `VectorSpaceRetrievalModel`
            similarity_fn: function instance that takes in two vectors 
                            and returns a similarity score e.g cosine_sim defined earlier
        """
        self.ret = initial_retrieval_fn
        self.vsrm = vsrm
        self.similarity_fn = similarity_fn
        self.vectorized_documents = vsrm.vectorize_documents()
        
        assert len(self.vectorized_documents) == len(doc_repr_2)
    
    def search(self, query, K=50):
        """
            First, retrieve the top K results using the retrieval function
            Then, re-rank the results using the VSRM instance
        """
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
##### Function check
bm25_search_2 = partial(bm25_search, index_set=2)
lsi_rerank = DenseRerankingModel(bm25_search_2, lsi, cosine_sim)
lda_rerank = DenseRerankingModel(bm25_search_2, lda, jenson_shannon_sim)
w2v_rerank = DenseRerankingModel(bm25_search_2, w2v, cosine_sim)
w2v_pretrained_rerank = DenseRerankingModel(bm25_search_2, w2v_pretrained, cosine_sim)
d2v_rerank = DenseRerankingModel(bm25_search_2, d2v, cosine_sim)

##### 

\#### Please do not change this. This cell is used for grading.

---
Now, let us time the new search functions:

In [ ]:
query = "algebraic functions"
print("BM25: ")
%timeit bm25_search(query, 2)
print("LSI: ")
%timeit lsi_rerank.search(query)
print("LDA: ")
%timeit lda_rerank.search(query)
print("W2V: ")
%timeit w2v_rerank.search(query)
print("W2V(Pretrained): ")
%timeit w2v_pretrained_rerank.search(query)
print("D2V:")
%timeit d2v_rerank.search(query)

---
As you can see, it is much faster (but BM25 is still orders of magnitude faster).

---
## Section 10: Evaluation & Analysis (30 points) <a class="anchor" id="reranking_eval"></a>

[Back to Part 2](#part2)

[Previously](#evaluation) we have implemented some evaluation metrics and used them for measuring the ranking performance of term-based IR algorithms. In this section, we will do the same for semantic methods, both with and without re-ranking.

### Section 10.1: Plot (10 points)

First, gather the results. The results should consider the index set, the different search functions and different metrics. Plot the results in bar charts, per metric, with clear labels.

Then, gather only the re-ranking models, and plot and compare them with the results obtained in part 1 (only index set 2).

In [ ]:
list_of_sem_search_fns = [
    ("lda", drm_lda.search),
    ("lsi", drm_lsi.search),
    ("w2v", drm_w2v.search),
    ("w2v_pretrained", drm_w2v_pretrained.search),
    ("d2v", drm_d2v.search),
    ("lsi_rr", lsi_rerank.search),
    ("lda_rr", lda_rerank.search),
    ("w2v_rr", w2v_rerank.search),
    ("w2v_pretrained_rr", w2v_pretrained_rerank.search),
    ("d2v_rr", d2v_rerank.search),
    
]

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Section 10.2: Summary (20 points)

Your summary should compare methods from Part 1 and Part 2 (only for index set 2). State what you expected to see in the results, followed by either supporting evidence *or* justify why the results did not support your expectations. Consider the availability of data, scalability, domain/type of data, etc.

YOUR ANSWER HERE